In [7]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
   #Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    #  Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    #  Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    #  Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")


      #Project
project = {
    "name": "Hotel Reservation Management System",
    "Description": "A system that allows customers to make online reservations, view room availability, and receive automatic confirmations.",
    "guidelines": [
        "Intuitive user experience",
        "Secure payments",
        "Scalability",
        "Continuous availability",
        "Multilanguage support"
    ],
    "requirements": [
        {"id": 1, "description": "As a customer, I want to search for available rooms to make an online reservation."},
        {"id": 2, "description": "As a customer, I want to receive reservation confirmations by email and SMS."},
        {"id": 3, "description": "As a customer, I want to cancel or modify my reservation online."},
        {"id": 4, "description": "As an administrator, I want to view an updated room availability calendar."},
        {"id": 5, "description": "As a manager, I want occupancy and reservation reports."},
        {"id": 6, "description": "As a customer, I want an interface in my preferred language."},
        {"id": 7, "description": "As a manager, I want monthly and annual revenue reports."},
        {"id": 8, "description": "As a user, I want to save personal notes during the course."},
        {"id": 9, "description": "As a customer, I want to see photos and detailed descriptions of the rooms."},
        {"id": 10, "description": "As a customer, I want early check-in and late check-out options."},
        {"id": 11, "description": "As an administrator, I want maintenance alerts for specific rooms."},
        {"id": 12, "description": "As a customer, I want to add additional services like transport or spa."},
        {"id": 13, "description": "As an administrator, I want to manage the inventory of hotel services and products."},
        {"id": 14, "description": "As a customer, I want to see reviews from other guests."},
        {"id": 15, "description": "As a manager, I want reports on the most requested additional services."},
        {"id": 16, "description": "As an administrator, I want access to a history of all previous reservations."},
        {"id": 17, "description": "As a customer, I want to receive notifications about special offers or discounts."},
        {"id": 18, "description": "As an administrator, I want to assign specific roles to hotel staff."},
        {"id": 19, "description": "As a customer, I want to be able to pay in different currencies according to my preference."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Hotel Reservation Management System
Guidelines: ['Intuitive user experience', 'Secure payments', 'Scalability', 'Continuous availability', 'Multilanguage support']
Dimensiones asociadas (7): ['Technical Context', 'Operational Benefits', 'System-related Benefits', 'Costs', 'Business Context', 'Stakeholder Benefits', 'Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  52: Contribution to user task
  2: Linkage to overall system goals
  37: Relevance to stakeholders' goals
  172: Time to market
  183: Integration to external systems
  236: Dependencies due to downstream activities
  197: Experience on the area
  174: Impact on Maintenance
  198: Experience on development methodology
  177: Impact on long-term Evolution
  215: Sufficient detail in specification
  56: Cost reduction of IT operation

Requisitos con sus subdimensiones y PriorityScore:

1. A

In [8]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")


      #Project 2
project = {
    "name": "E-learning Platform for Companies",
    "Description": "A platform that facilitates online training for employees, with course management, assessments, and progress tracking.",
    "guidelines": [
        "Ease of use",
        "Course flexibility",
        "Integration with video conferencing",
        "Scalability",
        "Data security"
    ],
    "requirements": [
        {"id": 1, "description": "As an employee, I want to enroll in courses and complete evaluations online."},
        {"id": 2, "description": "As an instructor, I want to monitor employee progress."},
        {"id": 3, "description": "As an administrator, I want employee performance reports."},
        {"id": 4, "description": "As a user, I want secure access to video conferences."},
        {"id": 5, "description": "As a manager, I want reports on course effectiveness."},
        {"id": 6, "description": "As a user, I want to receive course completion certificates."},
        {"id": 7, "description": "As an instructor, I want to schedule live or recorded sessions."},
        {"id": 8, "description": "As a user, I want to save personal notes during the course."},
        {"id": 9, "description": "As an administrator, I want to manage access permissions for specific courses."},
        {"id": 10, "description": "As an employee, I want to review my completed course history."},
        {"id": 11, "description": "As a manager, I want to see statistics on the most popular courses."},
        {"id": 12, "description": "As a user, I want to access courses from any device."},
        {"id": 13, "description": "As an administrator, I want to customize content according to the audience."},
        {"id": 14, "description": "As a user, I want to participate in course discussion forums."},
        {"id": 15, "description": "As an administrator, I want to restrict access to certain materials."},
        {"id": 16, "description": "As an employee, I want to receive deadline reminders."},
        {"id": 17, "description": "As a manager, I want to compare performance across different teams."},
        {"id": 18, "description": "As an administrator, I want to manage evaluations and grading automatically."},
        {"id": 19, "description": "As a user, I want to share my achievements on social media if I wish."}
    ]
}


associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: E-learning Platform for Companies
Guidelines: ['Ease of use', 'Course flexibility', 'Integration with video conferencing', 'Scalability', 'Data security']
Dimensiones asociadas (7): ['Risks', 'Technical Context', 'Business Context', 'System-related Benefits', 'Operational Benefits', 'Market-related Benefits', 'Stakeholder Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  52: Contribution to user task
  183: Integration to external systems
  2: Linkage to overall system goals
  37: Relevance to stakeholders' goals
  172: Time to market
  236: Dependencies due to downstream activities
  197: Experience on the area
  110: Technical risk in current system
  111: Technical risk in proposed system
  215: Sufficient detail in specification
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements

Requisitos con sus 

In [9]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")


# PJT-003
project = {
    "name": "Employee Wellness Application",
    "Description": "A platform for raising funds for social projects, allowing users to create campaigns and make online donations.",
    "guidelines": [
        "Well-being promotion",
        "Personalized reminders",
        "Integration with wearables",
        "Data protection",
        "User-friendly interface"
    ],
    "requirements": [
        {"id": 1, "description": "As an employee, I want to log my daily physical activity."},
        {"id": 2, "description": "As an employee, I want to receive reminders to rest."},
        {"id": 3, "description": "As a wellness administrator, I want to see app usage reports."},
        {"id": 4, "description": "As a user, I want to sync my wearable with the app."},
        {"id": 5, "description": "As an administrator, I want to protect health data."},
        {"id": 6, "description": "As a user, I want to receive alerts for prolonged physical activity."},
        {"id": 7, "description": "As a user, I want to log my daily stress level."},
        {"id": 8, "description": "As an administrator, I want to compare physical activity by department."},
        {"id": 9, "description": "As a user, I want to create weekly wellness goals."},
        {"id": 10, "description": "As an administrator, I want to see wellness trends in the company."},
        {"id": 11, "description": "As a user, I want to participate in group wellness challenges."},
        {"id": 12, "description": "As a user, I want access to health tips based on my activity."},
        {"id": 13, "description": "As an administrator, I want notifications about completed challenges."},
        {"id": 14, "description": "As a user, I want to receive monthly reports on my progress."},
        {"id": 15, "description": "As a user, I want to receive stretching and exercise recommendations."},
        {"id": 16, "description": "As an administrator, I want to manage access to external wellness resources."},
        {"id": 17, "description": "As a user, I want to receive active break alerts."},
        {"id": 18, "description": "As a user, I want to customize activity reminders."},
        {"id": 19, "description": "As an administrator, I want access to metrics on overall well-being."}
    ]
}


associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Employee Wellness Application
Guidelines: ['Well-being promotion', 'Personalized reminders', 'Integration with wearables', 'Data protection', 'User-friendly interface']
Dimensiones asociadas (7): ['Operational Benefits', 'Technical Context', 'Market-related Benefits', 'Stakeholder Benefits', 'Costs', 'Risks', 'Business Context']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  52: Contribution to user task
  183: Integration to external systems
  37: Relevance to stakeholders' goals
  172: Time to market
  236: Dependencies due to downstream activities
  228: Number of use cases inherited by this use case
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  110: Technical risk in current system
  111: Tech

In [10]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")


# PJT-004
project = {
    "name": "Inventory Management System for E-commerce",
    "Description": "A system to track stock levels, manage product listings, and automate restocking processes for an online store.",
    "guidelines": [
        "Real-time updates",
        "Low latency",
        "User-friendly dashboard",
        "Supplier integration",
        "Scalable architecture"
    ],
    "requirements": [
        {"id": 1, "description": "As a store manager, I want to view real-time stock levels."},
        {"id": 2, "description": "As an admin, I want automatic low-stock alerts."},
        {"id": 3, "description": "As a user, I want to add new products easily."},
        {"id": 4, "description": "As an admin, I want to integrate supplier catalogs."},
        {"id": 5, "description": "As a manager, I want to generate inventory reports."},
        {"id": 6, "description": "As a user, I want to search products by SKU."},
        {"id": 7, "description": "As an admin, I want to update stock counts in bulk."},
        {"id": 8, "description": "As a manager, I want forecasts for restocking."},
        {"id": 9, "description": "As a user, I want to view product history."},
        {"id": 10, "description": "As an admin, I want role-based access control."},
        {"id": 11, "description": "As a user, I want barcode scanning support."},
        {"id": 12, "description": "As a manager, I want supplier performance metrics."},
        {"id": 13, "description": "As an admin, I want automated reorder point calculation."},
        {"id": 14, "description": "As a user, I want exportable CSV reports."},
        {"id": 15, "description": "As a manager, I want alerts on expired products."},
        {"id": 16, "description": "As a user, I want to filter products by category."},
        {"id": 17, "description": "As a manager, I want audit logs of inventory changes."},
        {"id": 18, "description": "As an admin, I want to schedule bulk updates."},
        {"id": 19, "description": "As a user, I want push notifications for critical events."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Inventory Management System for E-commerce
Guidelines: ['Real-time updates', 'Low latency', 'User-friendly dashboard', 'Supplier integration', 'Scalable architecture']
Dimensiones asociadas (7): ['Business Context', 'Operational Benefits', 'Costs', 'Risks', 'Technical Context', 'Stakeholder Benefits', 'Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  52: Contribution to user task
  172: Time to market
  183: Integration to external systems
  37: Relevance to stakeholders' goals
  228: Number of use cases inherited by this use case
  236: Dependencies due to downstream activities
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  110: Technical risk in current system
  111: Technical risk in pr

In [11]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")


# PJT-005
project = {
    "name": "Telemedicine Application for Medical Consultations",
    "Description": "An app enabling remote consultations with doctors, including video calls, prescription sharing, and patient history tracking.",
    "guidelines": [
        "HIPAA compliance",
        "High-quality video",
        "Prescription security",
        "Easy appointment scheduling",
        "Cross-platform support"
    ],
    "requirements": [
        {"id": 1, "description": "As a patient, I want to book video consultations."},
        {"id": 2, "description": "As a doctor, I want to access patient history."},
        {"id": 3, "description": "As a patient, I want to receive prescriptions digitally."},
        {"id": 4, "description": "As an admin, I want appointment reminders."},
        {"id": 5, "description": "As a patient, I want secure messaging with doctors."},
        {"id": 6, "description": "As a doctor, I want to write consultation notes."},
        {"id": 7, "description": "As a patient, I want to upload medical records."},
        {"id": 8, "description": "As an admin, I want usage and performance reports."},
        {"id": 9, "description": "As a patient, I want to rate consultations."},
        {"id": 10, "description": "As a doctor, I want to prescribe medication electronically."},
        {"id": 11, "description": "As a patient, I want multi-language support."},
        {"id": 12, "description": "As an admin, I want to manage doctor availability."},
        {"id": 13, "description": "As a patient, I want to see billing history."},
        {"id": 14, "description": "As a doctor, I want notifications for new consultations."},
        {"id": 15, "description": "As a patient, I want follow-up reminders."},
        {"id": 16, "description": "As an admin, I want data export capabilities."},
        {"id": 17, "description": "As a doctor, I want to refer patients to specialists."},
        {"id": 18, "description": "As a patient, I want a summary of past consultations."},
        {"id": 19, "description": "As an admin, I want system health monitoring."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Telemedicine Application for Medical Consultations
Guidelines: ['HIPAA compliance', 'High-quality video', 'Prescription security', 'Easy appointment scheduling', 'Cross-platform support']
Dimensiones asociadas (7): ['Operational Benefits', 'Costs', 'Business Context', 'System-related Benefits', 'Risks', 'Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  2: Linkage to overall system goals
  52: Contribution to user task
  172: Time to market
  183: Integration to external systems
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  110: Technical risk in current system
  111: Technical risk in proposed system
  56: Cost reduction of IT operation
  186: IT departments technical guidelines
  53: Support for main usage scenario
  77: Control and follow-up

Requisitos con sus subdimensiones y PriorityScore

In [12]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")


# PJT-006
project = {
    "name": "Logistics System for Transport Routes",
    "Description": "A system to optimize routing and scheduling for transport fleets, minimizing costs and delivery times.",
    "guidelines": [
        "Route optimization",
        "Real-time tracking",
        "Fuel efficiency",
        "Scalability",
        "ETA accuracy"
    ],
    "requirements": [
        {"id": 1, "description": "As a dispatcher, I want optimized route suggestions."},
        {"id": 2, "description": "As a driver, I want live GPS tracking."},
        {"id": 3, "description": "As a manager, I want cost analysis reports."},
        {"id": 4, "description": "As a dispatcher, I want dynamic rerouting."},
        {"id": 5, "description": "As a driver, I want fuel consumption stats."},
        {"id": 6, "description": "As a manager, I want delivery time forecasts."},
        {"id": 7, "description": "As a dispatcher, I want to assign vehicles to routes."},
        {"id": 8, "description": "As a manager, I want fleet utilization metrics."},
        {"id": 9, "description": "As a driver, I want offline route access."},
        {"id": 10, "description": "As a dispatcher, I want alerts for delays."},
        {"id": 11, "description": "As a manager, I want maintenance scheduling."},
        {"id": 12, "description": "As a driver, I want traffic condition updates."},
        {"id": 13, "description": "As a dispatcher, I want to manage driver shifts."},
        {"id": 14, "description": "As a manager, I want historical delivery performance."},
        {"id": 15, "description": "As a dispatcher, I want to prioritize deliveries."},
        {"id": 16, "description": "As a driver, I want incident reporting."},
        {"id": 17, "description": "As a manager, I want to compare route efficiency over time."},
        {"id": 18, "description": "As a dispatcher, I want integration with warehouse systems."},
        {"id": 19, "description": "As a driver, I want voice-guided navigation."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Logistics System for Transport Routes
Guidelines: ['Route optimization', 'Real-time tracking', 'Fuel efficiency', 'Scalability', 'ETA accuracy']
Dimensiones asociadas (7): ['System-related Benefits', 'Business Context', 'Customer Benefits', 'Operational Benefits', 'Costs', 'Risks', 'Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  110: Technical risk in current system
  111: Technical risk in proposed system
  174: Impact on Maintenance
  105: Impact on essential non-functional requirements
  172: Time to market
  177: Impact on long-term Evolution
  183: Integration to external systems
  2: Linkage to overall system goals
  52: Contribution to user task
  56: Cost reduction of IT operation
  47: Efficiency gains for customer
  53: Support for main usage scenario
  100: Cost for fixing defects
  155: Part time team member use

Requisitos con sus subdimensiones y PriorityScore:

1. As a dispatcher, I want optimized route suggestions.
    

In [13]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-007
project = {
    "name": "Crowdfunding Platform for Social Projects",
    "Description": "A platform for raising funds for social projects, allowing users to create campaigns and make online donations.",
    "guidelines": [
        "Campaign customization",
        "Secure payments",
        "Social sharing",
        "Transparent funding",
        "User verification"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to create a fundraising campaign."},
        {"id": 2, "description": "As a donor, I want to browse active campaigns."},
        {"id": 3, "description": "As a user, I want to share campaigns on social media."},
        {"id": 4, "description": "As an admin, I want to verify user identities."},
        {"id": 5, "description": "As a donor, I want to make secure payments."},
        {"id": 6, "description": "As a user, I want progress bars on campaign pages."},
        {"id": 7, "description": "As an admin, I want reporting on donation trends."},
        {"id": 8, "description": "As a user, I want to set donation goals."},
        {"id": 9, "description": "As a donor, I want to receive tax receipts."},
        {"id": 10, "description": "As an admin, I want to manage campaign categories."},
        {"id": 11, "description": "As a user, I want to edit my campaign description."},
        {"id": 12, "description": "As a donor, I want to schedule recurring donations."},
        {"id": 13, "description": "As an admin, I want to flag fraudulent campaigns."},
        {"id": 14, "description": "As a user, I want notifications on campaign milestones."},
        {"id": 15, "description": "As a donor, I want to comment on campaigns."},
        {"id": 16, "description": "As an admin, I want to export donor lists."},
        {"id": 17, "description": "As a user, I want to embed videos in my campaign."},
        {"id": 18, "description": "As a donor, I want to view campaign updates."},
        {"id": 19, "description": "As an admin, I want to generate campaign performance reports."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Crowdfunding Platform for Social Projects
Guidelines: ['Campaign customization', 'Secure payments', 'Social sharing', 'Transparent funding', 'User verification']
Dimensiones asociadas (7): ['Technical Context', 'Stakeholder Benefits', 'Operational Benefits', 'Costs', 'Risks', 'Benefits', 'System-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  52: Contribution to user task
  105: Impact on essential non-functional requirements
  197: Experience on the area
  198: Experience on development methodology
  2: Linkage to overall system goals
  37: Relevance to stakeholders' goals
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  228: Number of use cases inherited by this use case
  236: Dependencies due to downstream acti

In [14]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-008
project = {
    "name": "Personal Finance Application",
    "Description": "An app to track income and expenses, set budgets, and visualize financial health.",
    "guidelines": [
        "Security",
        "Data encryption",
        "Budget alerts",
        "Cross-platform sync",
        "Intuitive UI"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to log income and expenses."},
        {"id": 2, "description": "As a user, I want to set monthly budgets."},
        {"id": 3, "description": "As an admin, I want to manage currency rates."},
        {"id": 4, "description": "As a user, I want spending trend charts."},
        {"id": 5, "description": "As a user, I want notifications for overspending."},
        {"id": 6, "description": "As a user, I want to categorize transactions."},
        {"id": 7, "description": "As an admin, I want to export financial data."},
        {"id": 8, "description": "As a user, I want cross-device synchronization."},
        {"id": 9, "description": "As a user, I want reminders for upcoming bills."},
        {"id": 10, "description": "As an admin, I want to back up user data daily."},
        {"id": 11, "description": "As a user, I want to set savings goals."},
        {"id": 12, "description": "As a user, I want to link bank accounts."},
        {"id": 13, "description": "As an admin, I want to monitor system usage."},
        {"id": 14, "description": "As a user, I want expense tagging suggestions."},
        {"id": 15, "description": "As a user, I want to share summaries with an advisor."},
        {"id": 16, "description": "As an admin, I want role-based access control."},
        {"id": 17, "description": "As a user, I want currency conversion."},
        {"id": 18, "description": "As a user, I want biometric login options."},
        {"id": 19, "description": "As an admin, I want system health dashboards."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Personal Finance Application
Guidelines: ['Security', 'Data encryption', 'Budget alerts', 'Cross-platform sync', 'Intuitive UI']
Dimensiones asociadas (7): ['Risks', 'System-related Benefits', 'Benefits', 'Market-related Benefits', 'Project-related Benefits', 'Stakeholder Benefits', 'Financial Benefits']

Subdimensiones esenciales (ID: Nombre):
  110: Technical risk in current system
  111: Technical risk in proposed system
  35: End user value
  2: Linkage to overall system goals
  3: Ease of use
  155: Part time team member use
  26: Feature influence on buying decision
  31: Importance to release goal
  37: Relevance to stakeholders' goals
  39: Return on investment
  105: Impact on essential non-functional requirements
  30: Relevance to project success
  33: Feature contribution to project vision
  36: Value creation for developer
  109: Loss of confidential data

Requisitos con sus subdimensiones y PriorityScore:

1. As a user, I want to log income and expenses.
    - 

In [15]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-009
project = {
    "name": "Energy Monitoring System for Buildings",
    "Description": "A system to monitor and analyze energy consumption in real estate, with alerts for inefficiencies.",
    "guidelines": [
        "Real-time analytics",
        "Low power usage",
        "Scalable sensors",
        "User-friendly reporting",
        "Alert notifications"
    ],
    "requirements": [
        {"id": 1, "description": "As a facility manager, I want real-time energy dashboards."},
        {"id": 2, "description": "As a user, I want alerts for high consumption."},
        {"id": 3, "description": "As an admin, I want to configure sensor networks."},
        {"id": 4, "description": "As a user, I want historical usage graphs."},
        {"id": 5, "description": "As an admin, I want exportable CSV data."},
        {"id": 6, "description": "As a user, I want mobile access."},
        {"id": 7, "description": "As an admin, I want sensor health reports."},
        {"id": 8, "description": "As a user, I want customizable alert thresholds."},
        {"id": 9, "description": "As an admin, I want integration with building management systems."},
        {"id": 10, "description": "As a user, I want consumption forecasts."},
        {"id": 11, "description": "As an admin, I want role-based dashboards."},
        {"id": 12, "description": "As a user, I want energy-saving tips."},
        {"id": 13, "description": "As an admin, I want audit logs."},
        {"id": 14, "description": "As a user, I want multi-building support."},
        {"id": 15, "description": "As a user, I want push notifications for anomalies."},
        {"id": 16, "description": "As an admin, I want schedule-based reporting."},
        {"id": 17, "description": "As a user, I want drill-down by device."},
        {"id": 18, "description": "As an admin, I want integration with ERP systems."},
        {"id": 19, "description": "As a user, I want exportable PDF reports."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Energy Monitoring System for Buildings
Guidelines: ['Real-time analytics', 'Low power usage', 'Scalable sensors', 'User-friendly reporting', 'Alert notifications']
Dimensiones asociadas (7): ['Technical Context', 'Operational Benefits', 'Costs', 'Business Context', 'Stakeholder Benefits', 'Risks', 'Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  228: Number of use cases inherited by this use case
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  110: Technical risk in current system
  111: Technical risk in proposed system
  215: Sufficient detail in specification
  52: Contribution to user task
  155: Part time team member use
  35: End user value
  99: User acceptance testing cost

Requisi

In [16]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-010
project = {
    "name": "Networking Portal for Professionals",
    "Description": "A portal that connects professionals, facilitates networking events, and shares industry news.",
    "guidelines": [
        "Profile customization",
        "Event integration",
        "News feed",
        "Secure messaging",
        "Mobile responsiveness"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to create a professional profile."},
        {"id": 2, "description": "As a user, I want to join networking events."},
        {"id": 3, "description": "As a user, I want to follow industry news."},
        {"id": 4, "description": "As a user, I want secure one-to-one messaging."},
        {"id": 5, "description": "As a user, I want to endorse connections."},
        {"id": 6, "description": "As an admin, I want to curate news articles."},
        {"id": 7, "description": "As a user, I want event reminders."},
        {"id": 8, "description": "As an admin, I want user activity reports."},
        {"id": 9, "description": "As a user, I want advanced search filters."},
        {"id": 10, "description": "As a user, I want mobile push notifications."},
        {"id": 11, "description": "As an admin, I want to manage event listings."},
        {"id": 12, "description": "As a user, I want to connect with peers."},
        {"id": 13, "description": "As an admin, I want analytics on engagement."},
        {"id": 14, "description": "As a user, I want to share articles."},
        {"id": 15, "description": "As a user, I want group discussion forums."},
        {"id": 16, "description": "As a user, I want to tag skills on my profile."},
        {"id": 17, "description": "As an admin, I want to manage user roles."},
        {"id": 18, "description": "As a user, I want to sync LinkedIn contacts."},
        {"id": 19, "description": "As an admin, I want system performance monitoring."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Networking Portal for Professionals
Guidelines: ['Profile customization', 'Event integration', 'News feed', 'Secure messaging', 'Mobile responsiveness']
Dimensiones asociadas (7): ['Operational Benefits', 'Costs', 'Business Context', 'Benefits', 'System-related Benefits', 'Market-related Benefits', 'Project-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  52: Contribution to user task
  3: Ease of use
  188: Frequency of use
  2: Linkage to overall system goals
  26: Feature influence on buying decision
  99: User acceptance testing cost
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  183: Integration to external systems
  185: Use case weight
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  172: Time to market

Requisitos con sus subdimensiones y PriorityScore:

1. As a user, I want to create a professional profile.
    - 52: Contribution to us

In [17]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-011
project = {
    "name": "Personalized Online Learning Platform",
    "Description": "A system that adapts learning paths to individual students, with analytics and progress tracking.",
    "guidelines": [
        "Adaptive content",
        "Progress analytics",
        "Interactive quizzes",
        "Scalable infrastructure",
        "Multimedia support"
    ],
    "requirements": [
        {"id": 1, "description": "As a student, I want personalized learning paths."},
        {"id": 2, "description": "As an instructor, I want to view student analytics."},
        {"id": 3, "description": "As a user, I want interactive quizzes."},
        {"id": 4, "description": "As an admin, I want to manage content adaptivity rules."},
        {"id": 5, "description": "As a student, I want progress notifications."},
        {"id": 6, "description": "As an instructor, I want to assign remedial modules."},
        {"id": 7, "description": "As a user, I want multimedia lessons."},
        {"id": 8, "description": "As an admin, I want real-time scalability."},
        {"id": 9, "description": "As a student, I want to set learning goals."},
        {"id": 10, "description": "As an instructor, I want to create branching scenarios."},
        {"id": 11, "description": "As a user, I want peer discussion boards."},
        {"id": 12, "description": "As an admin, I want A/B testing capabilities."},
        {"id": 13, "description": "As a student, I want mobile access."},
        {"id": 14, "description": "As an instructor, I want notifications for at-risk students."},
        {"id": 15, "description": "As a user, I want downloadable lesson materials."},
        {"id": 16, "description": "As an admin, I want backup and restore tools."},
        {"id": 17, "description": "As a student, I want progress export to CSV."},
        {"id": 18, "description": "As a user, I want to bookmark lessons."},
        {"id": 19, "description": "As an instructor, I want customized assessment templates."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Personalized Online Learning Platform
Guidelines: ['Adaptive content', 'Progress analytics', 'Interactive quizzes', 'Scalable infrastructure', 'Multimedia support']
Dimensiones asociadas (7): ['Business Context', 'Operational Benefits', 'Costs', 'Benefits', 'System-related Benefits', 'Market-related Benefits', 'Project-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  52: Contribution to user task
  172: Time to market
  2: Linkage to overall system goals
  31: Importance to release goal
  30: Relevance to project success
  33: Feature contribution to project vision
  183: Integration to external systems
  26: Feature influence on buying decision
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  99: User acceptance testing cost
  3: Ease of use
  185: Use case weight
  188: Frequency of use

Requisitos con sus subdimensiones y PriorityScore:

1. As a student, I want personalized learning paths.
    - 2: Linkage t

In [18]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-012
project = {
    "name": "Hiring Platform for Companies",
    "Description": "A professional networking platform that facilitates user connections and job opportunity searches.",
    "guidelines": [
        "Efficient job posting",
        "Information security",
        "Access to relevant profiles",
        "Hiring reports",
        "User-friendly interface"
    ],
    "requirements": [
        {"id": 1, "description": "As an employer, I want to post job openings quickly and efficiently."},
        {"id": 2, "description": "As a candidate, I want to easily apply to the job openings that interest me."},
        {"id": 3, "description": "As an administrator, I want to manage access permissions to the platform."},
        {"id": 4, "description": "As an employer, I want to receive notifications of new applications to my job postings."},
        {"id": 5, "description": "As a candidate, I want to save interesting job openings to apply later."},
        {"id": 6, "description": "As an employer, I want to view a detailed profile of the candidates."},
        {"id": 7, "description": "As an administrator, I want reports on candidates per job opening."},
        {"id": 8, "description": "As a candidate, I want to receive alerts about new relevant job openings."},
        {"id": 9, "description": "As an employer, I want to contact candidates through the platform."},
        {"id": 10, "description": "As an administrator, I want to manage job promotion campaigns."},
        {"id": 11, "description": "As a candidate, I want to receive job recommendations based on my profile."},
        {"id": 12, "description": "As an employer, I want to filter candidates based on specific skills."},
        {"id": 13, "description": "As an administrator, I want statistics on the most popular job openings."},
        {"id": 14, "description": "As a candidate, I want to receive alerts for upcoming stages in the selection process."},
        {"id": 15, "description": "As an employer, I want to evaluate candidates directly on the platform."},
        {"id": 16, "description": "As an administrator, I want hiring time reports by job opening."},
        {"id": 17, "description": "As a candidate, I want to easily update my profile and professional experience."},
        {"id": 18, "description": "As an employer, I want to view the background of each selected candidate."},
        {"id": 19, "description": "As an administrator, I want to generate user satisfaction reports for the recruitment process."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Hiring Platform for Companies
Guidelines: ['Efficient job posting', 'Information security', 'Access to relevant profiles', 'Hiring reports', 'User-friendly interface']
Dimensiones asociadas (7): ['Technical Context', 'Project-related Benefits', 'Operational Benefits', 'System-related Benefits', 'Stakeholder Benefits', 'Strategy-related Benefits', 'Business Context']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  52: Contribution to user task
  44: Criticality to mission success
  172: Time to market
  2: Linkage to overall system goals
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  37: Relevance to stakeholders' goals
  43: Importance to business goals
  183: Integration to external systems
  236: Dependencies due to downstream activities
  197: Experience on the area

Requisitos con sus su

In [19]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-013
project = {
    "name": "Task and Productivity Application",
    "Description": "A system that allows customers to make restaurant reservations and manage takeout orders.",
    "guidelines": [
        "Task organization",
        "Cross-platform access",
        "Reminder personalization",
        "Productivity analysis",
        "Data security"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to create tasks and assign due dates to each one."},
        {"id": 2, "description": "As a user, I want to receive reminders before each task's deadline."},
        {"id": 3, "description": "As an administrator, I want usage statistics to improve the platform."},
        {"id": 4, "description": "As a user, I want to group tasks under a project."},
        {"id": 5, "description": "As a user, I want to access my tasks from different devices."},
        {"id": 6, "description": "As a user, I want to set priorities for each task."},
        {"id": 7, "description": "As a user, I want to receive reports on my weekly productivity."},
        {"id": 8, "description": "As a user, I want to see progress charts for each project."},
        {"id": 9, "description": "As an administrator, I want to manage access permissions for advanced features."},
        {"id": 10, "description": "As a user, I want to configure custom reminders according to my routine."},
        {"id": 11, "description": "As a user, I want to share specific projects with collaborators."},
        {"id": 12, "description": "As an administrator, I want reports on completed and ongoing tasks."},
        {"id": 13, "description": "As a user, I want to receive suggestions for organization and improving my productivity."},
        {"id": 14, "description": "As a user, I want to receive notifications about overdue tasks."},
        {"id": 15, "description": "As a user, I want access to a monthly summary of my productivity."},
        {"id": 16, "description": "As an administrator, I want statistics on active users on the platform."},
        {"id": 17, "description": "As a user, I want to customize the design and colors of the app."},
        {"id": 18, "description": "As a user, I want to set productivity goals and track them."},
        {"id": 19, "description": "As a user, I want to sync my tasks with my device's calendar."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Task and Productivity Application
Guidelines: ['Task organization', 'Cross-platform access', 'Reminder personalization', 'Productivity analysis', 'Data security']
Dimensiones asociadas (7): ['Risks', 'Operational Benefits', 'System-related Benefits', 'Costs', 'Technical Context', 'Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  52: Contribution to user task
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  228: Number of use cases inherited by this use case
  236: Dependencies due to downstream activities
  2: Linkage to overall system goals
  197: Experience on the area
  3: Ease of use
  213: Number of times use case appears
  229: Feature weight from use cases
  26: Feature influence on buying 

In [20]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-014
project = {
    "name": "Library Management System",
    "Description": "A route optimization system for transport companies, helping reduce costs and track fleet locations in real time.",
    "guidelines": [
        "Online catalog access",
        "Loan management",
        "Due date notifications",
        "Usage reports",
        "Data security"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to search for books in the online catalog."},
        {"id": 2, "description": "As a user, I want to reserve available books for pickup."},
        {"id": 3, "description": "As an administrator, I want to manage the library's book inventory."},
        {"id": 4, "description": "As a user, I want to receive notifications about loan due dates."},
        {"id": 5, "description": "As an administrator, I want reports on the most borrowed books."},
        {"id": 6, "description": "As a user, I want to see a book's availability before requesting it."},
        {"id": 7, "description": "As an administrator, I want to manage loan renewals."},
        {"id": 8, "description": "As a user, I want to suggest new titles for the library."},
        {"id": 9, "description": "As an administrator, I want reports on frequent users and their preferences."},
        {"id": 10, "description": "As a user, I want to receive alerts about recommended books."},
        {"id": 11, "description": "As an administrator, I want statistics on lost or damaged books."},
        {"id": 12, "description": "As a user, I want to renew my loans online."},
        {"id": 13, "description": "As an administrator, I want to manage access permissions to the system."},
        {"id": 14, "description": "As a user, I want to view reviews and ratings from other users about the books."},
        {"id": 15, "description": "As an administrator, I want a loan history for each user."},
        {"id": 16, "description": "As a user, I want to receive alerts when a reserved book becomes available."},
        {"id": 17, "description": "As an administrator, I want to generate usage reports for the library."},
        {"id": 18, "description": "As a user, I want to mark books as favorites for future reading."},
        {"id": 19, "description": "As an administrator, I want customization options in the system based on the library's needs."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Library Management System
Guidelines: ['Online catalog access', 'Loan management', 'Due date notifications', 'Usage reports', 'Data security']
Dimensiones asociadas (7): ['Technical Context', 'Operational Benefits', 'Risks', 'System-related Benefits', 'Business Context', 'Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  52: Contribution to user task
  2: Linkage to overall system goals
  236: Dependencies due to downstream activities
  172: Time to market
  183: Integration to external systems
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  228: Number of use cases inherited by this use case
  197: Experience on the area
  110: Technical risk in current system

Re

In [21]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-015
project = {
    "name": "Event Management System",
    "Description": "A platform that adapts educational content based on each user's progress and preferences, enhancing the learning experience.",
    "guidelines": [
        "Ease of use",
        "Confirmation notifications",
        "Real-time reservation management",
        "Secure payments",
        "Menu personalization"
    ],
    "requirements": [
        {"id": 1, "description": "As a customer, I want to book a table online to avoid waiting times."},
        {"id": 2, "description": "As an administrator, I want to manage table availability in real time."},
        {"id": 3, "description": "As a customer, I want to receive a confirmation of my reservation via SMS."},
        {"id": 4, "description": "As an administrator, I want occupancy and reservation reports by shift."},
        {"id": 5, "description": "As a customer, I want to place takeout orders through the app."},
        {"id": 6, "description": "As an administrator, I want to see a reservation history of frequent customers."},
        {"id": 7, "description": "As a customer, I want to be able to cancel or modify my reservation online."},
        {"id": 8, "description": "As an administrator, I want to manage the menu and prices flexibly."},
        {"id": 9, "description": "As a customer, I want to see recommendations for popular dishes before ordering."},
        {"id": 10, "description": "As an administrator, I want to manage table assignments to maximize restaurant capacity."},
        {"id": 11, "description": "As a customer, I want online payment options for my orders."},
        {"id": 12, "description": "As an administrator, I want reports on dish sales and consumption."},
        {"id": 13, "description": "As a customer, I want to receive alerts about restaurant discounts and promotions."},
        {"id": 14, "description": "As an administrator, I want app customization options for the customer experience."},
        {"id": 15, "description": "As a customer, I want to receive a notification when my order is ready."},
        {"id": 16, "description": "As an administrator, I want to manage customer reviews and ratings."},
        {"id": 17, "description": "As a customer, I want to see photos and descriptions of dishes on the menu."},
        {"id": 18, "description": "As an administrator, I want statistics on the best-selling dishes."},
        {"id": 19, "description": "As a customer, I want pairing recommendations for the selected dishes."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Event Management System
Guidelines: ['Ease of use', 'Confirmation notifications', 'Real-time reservation management', 'Secure payments', 'Menu personalization']
Dimensiones asociadas (7): ['Technical Context', 'Risks', 'Business Context', 'System-related Benefits', 'Operational Benefits', 'Market-related Benefits', 'Financial Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  172: Time to market
  2: Linkage to overall system goals
  52: Contribution to user task
  183: Integration to external systems
  236: Dependencies due to downstream activities
  197: Experience on the area
  198: Experience on development methodology
  26: Feature influence on buying decision
  39: Return on investment
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution

Requisitos con sus subdimensiones y Pri

In [22]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-016
project = {
    "name": "Educational Fundraising Platform",
    "Description": "A sales platform that enables retailers to manage inventory, sales, and product promotions online.",
    "guidelines": [
        "Inventory management",
        "Secure payments",
        "Stock notifications",
        "Sales reports",
        "Mobile accessibility"
    ],
    "requirements": [
        {"id": 1, "description": "As a retailer, I want to manage product inventory in real time."},
        {"id": 2, "description": "As an administrator, I want to receive notifications when inventory is low."},
        {"id": 3, "description": "As a customer, I want to see product availability before making a purchase."},
        {"id": 4, "description": "As a retailer, I want to easily update product prices and promotions."},
        {"id": 5, "description": "As an administrator, I want sales reports by product category."},
        {"id": 6, "description": "As a customer, I want online payment options for my purchases."},
        {"id": 7, "description": "As a retailer, I want to schedule seasonal promotions and discounts."},
        {"id": 8, "description": "As a customer, I want to receive notifications about discounts and offers."},
        {"id": 9, "description": "As an administrator, I want a sales history to forecast demand."},
        {"id": 10, "description": "As a customer, I want to see product descriptions and photos in the catalog."},
        {"id": 11, "description": "As a retailer, I want personalized shipping options depending on the product type."},
        {"id": 12, "description": "As an administrator, I want statistics on best-selling products."},
        {"id": 13, "description": "As a customer, I want to receive purchase confirmations by email and SMS."},
        {"id": 14, "description": "As a retailer, I want to manage product availability across different physical stores."},
        {"id": 15, "description": "As an administrator, I want to generate reports on losses due to damaged products."},
        {"id": 16, "description": "As a customer, I want to see product ratings from other users."},
        {"id": 17, "description": "As a retailer, I want to integrate the platform with social media for product promotion."},
        {"id": 18, "description": "As an administrator, I want to manage platform access based on user roles."},
        {"id": 19, "description": "As a customer, I want to receive product recommendations based on my purchase history."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Educational Fundraising Platform
Guidelines: ['Inventory management', 'Secure payments', 'Stock notifications', 'Sales reports', 'Mobile accessibility']
Dimensiones asociadas (7): ['Market-related Benefits', 'Operational Benefits', 'Risks', 'Benefits', 'System-related Benefits', 'Project-related Benefits', 'Stakeholder Benefits']

Subdimensiones esenciales (ID: Nombre):
  52: Contribution to user task
  2: Linkage to overall system goals
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  37: Relevance to stakeholders' goals
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  110: Technical risk in current system
  111: Technical risk in proposed system
  15: Product Market value
  28: Long term Product strategy
  18: Additional customer sales
  16: Customer loyalty
  19: Extra cost customer will spend

Requisitos con sus subdimensiones y PriorityScore:

1. As a ret

In [23]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-017
project = {
    "name": "Restaurant Reservation System",
    "Description": "A system for managing school transportation bookings and logistics, improving communication between parents and drivers.",
    "guidelines": [
        "Donation security",
        "Fundraising transparency",
        "Impact reports",
        "Social media sharing",
        "Cause adaptability"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to create a campaign to raise funds for educational projects."},
        {"id": 2, "description": "As a donor, I want to make contributions securely."},
        {"id": 3, "description": "As an administrator, I want reports on received donations."},
        {"id": 4, "description": "As a user, I want to receive updates on my campaign’s progress."},
        {"id": 5, "description": "As an administrator, I want options to share campaigns on social media."},
        {"id": 6, "description": "As a donor, I want to see the impact of my contributions on education."},
        {"id": 7, "description": "As a user, I want recurring donation options to support long-term causes."},
        {"id": 8, "description": "As an administrator, I want statistics on the most popular educational campaigns."},
        {"id": 9, "description": "As a user, I want to receive a contribution certificate."},
        {"id": 10, "description": "As a donor, I want to be notified about new educational campaigns."},
        {"id": 11, "description": "As an administrator, I want to manage access permissions by role."},
        {"id": 12, "description": "As a user, I want to receive impact reports for each campaign."},
        {"id": 13, "description": "As a donor, I want international payment options."},
        {"id": 14, "description": "As an administrator, I want to generate success reports for educational campaigns."},
        {"id": 15, "description": "As a user, I want to receive notifications about campaign milestones."},
        {"id": 16, "description": "As a donor, I want the option to donate anonymously."},
        {"id": 17, "description": "As a user, I want to receive alerts about campaigns that need more support."},
        {"id": 18, "description": "As an administrator, I want fundraising trend reports for educational causes."},
        {"id": 19, "description": "As a user, I want to customize my profile with my preferred causes."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Restaurant Reservation System
Guidelines: ['Donation security', 'Fundraising transparency', 'Impact reports', 'Social media sharing', 'Cause adaptability']
Dimensiones asociadas (7): ['Risks', 'Business Context', 'System-related Benefits', 'Benefits', 'Market-related Benefits', 'Project-related Benefits', 'Stakeholder Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  177: Impact on long-term Evolution
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  26: Feature influence on buying decision
  33: Feature contribution to project vision
  30: Relevance to project success
  2: Linkage to overall system goals
  31: Importance to release goal
  37: Relevance to stakeholders' goals
  172: Time to market
  183: Integration to external systems
  3: Ease of use
  188: Frequency of use
  155: Part time team member use

Requisitos con sus subdimensiones y PriorityScore:

1. As a user, I want to create a campaign t

In [24]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-018
project = {
    "name": "Habit Tracking Platform",
    "Description": "A platform that connects clients with lawyers for online legal consultations, facilitating communication and case tracking.",
    "guidelines": [
        "Secure communication",
        "Flexible payment options",
        "Access to personalized materials",
        "Progress review",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As a student, I want to search for qualified tutors in specific subjects."},
        {"id": 2, "description": "As a tutor, I want to manage my availability schedule for sessions."},
        {"id": 3, "description": "As an administrator, I want to verify the identity of tutors before registration."},
        {"id": 4, "description": "As a student, I want to receive tutor recommendations based on my academic needs."},
        {"id": 5, "description": "As a tutor, I want access to video conferencing tools within the platform."},
        {"id": 6, "description": "As a student, I want to receive notifications about my upcoming sessions."},
        {"id": 7, "description": "As an administrator, I want performance reports for tutors."},
        {"id": 8, "description": "As a tutor, I want to track and grade students' performance."},
        {"id": 9, "description": "As a student, I want access to recordings of my sessions."},
        {"id": 10, "description": "As an administrator, I want statistics on the most requested subjects."},
        {"id": 11, "description": "As a student, I want to rate and comment on my experience with tutors."},
        {"id": 12, "description": "As a tutor, I want options to upload additional educational materials."},
        {"id": 13, "description": "As a student, I want to receive a summary of my academic progress on the platform."},
        {"id": 14, "description": "As an administrator, I want options to offer discounts on tutoring services."},
        {"id": 15, "description": "As a student, I want to schedule my tutoring sessions based on my availability."},
        {"id": 16, "description": "As a tutor, I want to manage the necessary tools for each session."},
        {"id": 17, "description": "As an administrator, I want reports on user usage and satisfaction."},
        {"id": 18, "description": "As a student, I want to receive additional study recommendations."},
        {"id": 19, "description": "As an administrator, I want to customize the interface for different regions."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Habit Tracking Platform
Guidelines: ['Secure communication', 'Flexible payment options', 'Access to personalized materials', 'Progress review', 'Scalability']
Dimensiones asociadas (7): ['Project-related Benefits', 'System-related Benefits', 'Strategy-related Benefits', 'Operational Benefits', 'Business Context', 'Stakeholder Benefits', 'Technical Context']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  52: Contribution to user task
  2: Linkage to overall system goals
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  37: Relevance to stakeholders' goals
  43: Importance to business goals
  44: Criticality to mission success
  172: Time to market
  183: Integration to external systems
  236: Dependencies due to downstream activities
  197: Experience on the area

Requisitos con sus subdimensio

In [25]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-019
project = {
    "name": "Online Tutoring Platform",
    "Description": "A platform that allows coaches to create personalized training plans for their clients, with progress tracking and activity logging.",
    "guidelines": [
        "Efficient event organization",
        "Attendee registration",
        "Payment integration",
        "Personalized notifications",
        "Attendance reports"
    ],
    "requirements": [
        {"id": 1, "description": "As an organizer, I want to create and customize events according to the type of activity."},
        {"id": 2, "description": "As an attendee, I want to register for events easily."},
        {"id": 3, "description": "As an administrator, I want attendance reports for each event."},
        {"id": 4, "description": "As an organizer, I want to manage the guest list and attendance confirmations."},
        {"id": 5, "description": "As an attendee, I want to receive notifications about the event I registered for."},
        {"id": 6, "description": "As an administrator, I want to manage payment options for events."},
        {"id": 7, "description": "As an organizer, I want to assign roles and tasks to other team members."},
        {"id": 8, "description": "As an attendee, I want to view an event summary and agenda."},
        {"id": 9, "description": "As an administrator, I want occupancy reports for event spaces."},
        {"id": 10, "description": "As an organizer, I want to send post-event satisfaction surveys."},
        {"id": 11, "description": "As an attendee, I want to receive a registration confirmation via email and SMS."},
        {"id": 12, "description": "As an organizer, I want to manage and update the event agenda in real time."},
        {"id": 13, "description": "As an administrator, I want income and expense reports for the event."},
        {"id": 14, "description": "As an organizer, I want options to share event information on social media."},
        {"id": 15, "description": "As an attendee, I want to receive alerts about changes to the event agenda."},
        {"id": 16, "description": "As an administrator, I want to control access to activities based on ticket type."},
        {"id": 17, "description": "As an organizer, I want statistics on event effectiveness."},
        {"id": 18, "description": "As an attendee, I want to be able to evaluate the activities at the end of the event."},
        {"id": 19, "description": "As an organizer, I want to customize the platform design for each event."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Online Tutoring Platform
Guidelines: ['Efficient event organization', 'Attendee registration', 'Payment integration', 'Personalized notifications', 'Attendance reports']
Dimensiones asociadas (7): ['Operational Benefits', 'Costs', 'Business Context', 'Benefits', 'System-related Benefits', 'Market-related Benefits', 'Project-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  52: Contribution to user task
  172: Time to market
  183: Integration to external systems
  2: Linkage to overall system goals
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  26: Feature influence on buying decision
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  77: Control and follow-up
  53: Support for main usage scenario
  100: Cost for fixing defects
  175: Importance for product architecture

Requisitos con sus subdimensiones y PriorityScore:

1. As an organizer, I wan

In [26]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-020
project = {
    "name": "Sales Platform for Retailers",
    "Description": "A system that helps companies manage customer information and transaction history.",
    "guidelines": [
        "Habit personalization",
        "Automatic reminders",
        "Progress reports",
        "Data security",
        "Cross-platform access"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to create and customize my daily habits."},
        {"id": 2, "description": "As a user, I want to receive reminders for each habit."},
        {"id": 3, "description": "As an administrator, I want usage reports for the application."},
        {"id": 4, "description": "As a user, I want to view my progress in each habit on a chart."},
        {"id": 5, "description": "As a user, I want to receive suggestions for new habits based on my interests."},
        {"id": 6, "description": "As an administrator, I want to see statistics on the most common habits."},
        {"id": 7, "description": "As a user, I want to share my achievements on social media."},
        {"id": 8, "description": "As a user, I want to receive alerts for incomplete habits."},
        {"id": 9, "description": "As an administrator, I want to manage access permissions to the application."},
        {"id": 10, "description": "As a user, I want to visualize the weekly and monthly progress of each habit."},
        {"id": 11, "description": "As a user, I want to receive recommendations from other users with similar habits."},
        {"id": 12, "description": "As an administrator, I want customization options for the app based on region."},
        {"id": 13, "description": "As a user, I want to set goals for each habit and track them."},
        {"id": 14, "description": "As a user, I want access to a history of all my completed habits."},
        {"id": 15, "description": "As an administrator, I want user satisfaction reports."},
        {"id": 16, "description": "As a user, I want to receive daily motivational messages."},
        {"id": 17, "description": "As a user, I want to mark habits as favorites."},
        {"id": 18, "description": "As an administrator, I want to see healthy habit trends on the platform."},
        {"id": 19, "description": "As a user, I want to sync my habits with my device’s calendar."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Sales Platform for Retailers
Guidelines: ['Habit personalization', 'Automatic reminders', 'Progress reports', 'Data security', 'Cross-platform access']
Dimensiones asociadas (7): ['Risks', 'System-related Benefits', 'Benefits', 'Market-related Benefits', 'Project-related Benefits', 'Stakeholder Benefits', 'Financial Benefits']

Subdimensiones esenciales (ID: Nombre):
  2: Linkage to overall system goals
  31: Importance to release goal
  37: Relevance to stakeholders' goals
  30: Relevance to project success
  33: Feature contribution to project vision
  26: Feature influence on buying decision
  39: Return on investment
  105: Impact on essential non-functional requirements
  35: End user value
  3: Ease of use
  155: Part time team member use
  110: Technical risk in current system
  111: Technical risk in proposed system
  150: Impediment of attaining the requirement
  36: Value creation for developer

Requisitos con sus subdimensiones y PriorityScore:

1. As a user, I wa

In [27]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-021
project = {
    "name": "Inventory Management System for Clinics",
    "Description": "A system that allows restaurants to control ingredient inventory, optimize orders, and reduce waste.",
    "guidelines": [
        "Accurate stock control",
        "Low inventory notifications",
        "Usage reports",
        "Supplier integration",
        "Data security"
    ],
    "requirements": [
        {"id": 1, "description": "As an administrator, I want to view the medical supplies inventory in real time."},
        {"id": 2, "description": "As an administrator, I want to receive alerts when inventory is low."},
        {"id": 3, "description": "As a purchasing assistant, I want to automatically generate restocking orders."},
        {"id": 4, "description": "As an administrator, I want reports on the usage of medical supplies."},
        {"id": 5, "description": "As an administrator, I want to manage access permissions to the inventory system."},
        {"id": 6, "description": "As a purchasing assistant, I want to check supply availability in other clinics."},
        {"id": 7, "description": "As an administrator, I want reports on inventory costs."},
        {"id": 8, "description": "As a user, I want to receive notifications about product expiration dates."},
        {"id": 9, "description": "As an administrator, I want to view statistics on supply consumption by department."},
        {"id": 10, "description": "As a purchasing assistant, I want to track purchase orders."},
        {"id": 11, "description": "As an administrator, I want to generate reports on losses due to expired or damaged products."},
        {"id": 12, "description": "As a purchasing assistant, I want to integrate restocking orders with suppliers."},
        {"id": 13, "description": "As an administrator, I want to obtain a consumption history for each supply."},
        {"id": 14, "description": "As a purchasing assistant, I want to schedule restocking based on usage patterns."},
        {"id": 15, "description": "As an administrator, I want savings reports related to inventory management."},
        {"id": 16, "description": "As a purchasing assistant, I want options to perform physical inventories."},
        {"id": 17, "description": "As an administrator, I want to manage viewing permissions based on roles."},
        {"id": 18, "description": "As a purchasing assistant, I want to receive alerts about delayed deliveries."},
        {"id": 19, "description": "As an administrator, I want inventory reports in exportable format for analysis."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Inventory Management System for Clinics
Guidelines: ['Accurate stock control', 'Low inventory notifications', 'Usage reports', 'Supplier integration', 'Data security']
Dimensiones asociadas (7): ['Operational Benefits', 'Risks', 'Technical Context', 'System-related Benefits', 'Costs', 'Business Context', 'Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  2: Linkage to overall system goals
  52: Contribution to user task
  172: Time to market
  183: Integration to external systems
  236: Dependencies due to downstream activities
  110: Technical risk in current system
  111: Technical risk in proposed system
  215: Sufficient detail in specification
  197: Experience on the area
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution

Requisitos con sus subdimensiones y PriorityScore:



In [28]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-022
project = {
    "name": "Online Recruitment Platform",
    "Description": "An educational platform offering courses and resources on health topics, aimed at patients, caregivers, and healthcare professionals.",
    "guidelines": [
        "Secure communication",
        "Secure payments",
        "Access to consultation history",
        "Case progress notifications",
        "Case scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As a client, I want to search for lawyers specialized in different areas."},
        {"id": 2, "description": "As a lawyer, I want to manage my availability for online consultations."},
        {"id": 3, "description": "As an administrator, I want to verify the identity of lawyers before registration."},
        {"id": 4, "description": "As a client, I want to receive lawyer recommendations based on my case."},
        {"id": 5, "description": "As a lawyer, I want access to a video conferencing tool on the platform."},
        {"id": 6, "description": "As a client, I want to receive notifications about the progress of my case."},
        {"id": 7, "description": "As an administrator, I want reports on consultations by case type."},
        {"id": 8, "description": "As a lawyer, I want to record and update each client’s history."},
        {"id": 9, "description": "As a client, I want access to a history of my consultations."},
        {"id": 10, "description": "As an administrator, I want statistics on the most common case types on the platform."},
        {"id": 11, "description": "As a client, I want to be able to rate my experience with the lawyer."},
        {"id": 12, "description": "As a lawyer, I want to upload important documents for the cases."},
        {"id": 13, "description": "As a client, I want to receive a monthly summary of my case progress."},
        {"id": 14, "description": "As an administrator, I want to manage access permissions to the system."},
        {"id": 15, "description": "As a client, I want to schedule my consultations according to my availability."},
        {"id": 16, "description": "As a lawyer, I want access to tools to manage consultations."},
        {"id": 17, "description": "As an administrator, I want user satisfaction reports."},
        {"id": 18, "description": "As a client, I want to receive reminders about upcoming consultations."},
        {"id": 19, "description": "As a lawyer, I want to see a complete consultation history for each client."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Online Recruitment Platform
Guidelines: ['Secure communication', 'Secure payments', 'Access to consultation history', 'Case progress notifications', 'Case scalability']
Dimensiones asociadas (7): ['Technical Context', 'Project-related Benefits', 'Business Context', 'System-related Benefits', 'Strategy-related Benefits', 'Operational Benefits', 'Stakeholder Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  197: Experience on the area
  2: Linkage to overall system goals
  44: Criticality to mission success
  52: Contribution to user task
  198: Experience on development methodology
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  37: Relevance to stakeholders' goals
  43: Importance to business goals
  172: Time to market
  174: Impact on Maintenance

Requisitos con sus subdimensiones 

In [29]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-023
project = {
    "name": "Corporate Training Platform",
    "Description": "A system for managing the inventory of medical supplies in clinics, enabling stock control and restocking orders.",
    "guidelines": [
        "Personal data privacy",
        "Cross-platform access",
        "Session personalization",
        "Automatic reminders",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to log my mood daily to keep track."},
        {"id": 2, "description": "As a user, I want access to guided meditation sessions on different topics."},
        {"id": 3, "description": "As an administrator, I want usage reports by tool type."},
        {"id": 4, "description": "As a user, I want to receive daily reminders to record my mood."},
        {"id": 5, "description": "As a user, I want to set weekly wellness goals."},
        {"id": 6, "description": "As an administrator, I want to see statistics on the most used breathing exercises."},
        {"id": 7, "description": "As a user, I want access to personalized wellness tips."},
        {"id": 8, "description": "As a user, I want to receive alerts when my mood significantly drops."},
        {"id": 9, "description": "As an administrator, I want to manage access permissions for advanced features."},
        {"id": 10, "description": "As a user, I want to be able to share my achievements on social media."},
        {"id": 11, "description": "As a user, I want to receive suggestions for new wellness activities."},
        {"id": 12, "description": "As an administrator, I want app customization options by region."},
        {"id": 13, "description": "As a user, I want access to a full history of my mood records."},
        {"id": 14, "description": "As a user, I want to mark exercises as favorites to repeat them."},
        {"id": 15, "description": "As an administrator, I want user satisfaction reports."},
        {"id": 16, "description": "As a user, I want to receive daily motivational messages."},
        {"id": 17, "description": "As a user, I want to customize the themes of the meditations."},
        {"id": 18, "description": "As an administrator, I want to see wellness trends on the platform."},
        {"id": 19, "description": "As a user, I want to sync my progress with my device calendar."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Corporate Training Platform
Guidelines: ['Personal data privacy', 'Cross-platform access', 'Session personalization', 'Automatic reminders', 'Scalability']
Dimensiones asociadas (7): ['System-related Benefits', 'Risks', 'Benefits', 'Market-related Benefits', 'Project-related Benefits', 'Stakeholder Benefits', 'Financial Benefits']

Subdimensiones esenciales (ID: Nombre):
  2: Linkage to overall system goals
  31: Importance to release goal
  33: Feature contribution to project vision
  37: Relevance to stakeholders' goals
  30: Relevance to project success
  26: Feature influence on buying decision
  39: Return on investment
  105: Impact on essential non-functional requirements
  3: Ease of use
  35: End user value
  155: Part time team member use
  110: Technical risk in current system
  111: Technical risk in proposed system
  150: Impediment of attaining the requirement
  36: Value creation for developer

Requisitos con sus subdimensiones y PriorityScore:

1. As a user, 

In [30]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-024
project = {
    "name": "Mental Wellness Application",
    "Description": "A system that enables retailers to manage their inventory, place restocking orders, and view sales statistics.",
    "guidelines": [
        "Maintenance service optimization",
        "Expiration notifications",
        "Cost reports",
        "GPS integration",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As an administrator, I want to schedule maintenance services for each vehicle."},
        {"id": 2, "description": "As a manager, I want to receive notifications when maintenance is approaching."},
        {"id": 3, "description": "As an administrator, I want maintenance cost reports for each vehicle."},
        {"id": 4, "description": "As a manager, I want to view the service history of each vehicle."},
        {"id": 5, "description": "As an administrator, I want to assign maintenance tasks to the appropriate staff."},
        {"id": 6, "description": "As an administrator, I want to manage access permissions to the maintenance system."},
        {"id": 7, "description": "As a manager, I want to see monthly maintenance cost statistics."},
        {"id": 8, "description": "As an administrator, I want alerts for vehicles with recurring issues."},
        {"id": 9, "description": "As a manager, I want performance reports of vehicles based on their maintenance."},
        {"id": 10, "description": "As an administrator, I want to manage services performed by external workshops."},
        {"id": 11, "description": "As a manager, I want replacement part recommendations based on usage history."},
        {"id": 12, "description": "As an administrator, I want integration options with GPS systems."},
        {"id": 13, "description": "As a manager, I want to compare maintenance costs across different types of vehicles."},
        {"id": 14, "description": "As an administrator, I want to generate reports on downtime due to maintenance."},
        {"id": 15, "description": "As a manager, I want to identify vehicles that require urgent maintenance."},
        {"id": 16, "description": "As an administrator, I want to receive alerts for out-of-stock spare parts."},
        {"id": 17, "description": "As a manager, I want reports on fuel efficiency after maintenance."},
        {"id": 18, "description": "As an administrator, I want to manage the spare parts inventory in the system."},
        {"id": 19, "description": "As a manager, I want to receive a weekly summary of the fleet's maintenance status."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Mental Wellness Application
Guidelines: ['Maintenance service optimization', 'Expiration notifications', 'Cost reports', 'GPS integration', 'Scalability']
Dimensiones asociadas (7): ['Costs', 'Operational Benefits', 'Risks', 'Business Context', 'System-related Benefits', 'Market-related Benefits', 'Penalties']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  160: Detraction from product's value
  183: Integration to external systems
  2: Linkage to overall system goals
  110: Technical risk in current system
  111: Technical risk in proposed system
  52: Contribution to user task
  172: Time to market
  159: Penalty in operation
  174: Impact on Maintenance
  53: Support for main usage scenario
  108: Risk of maintenance
  90: Maintenance cost
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements

Requisitos con sus subdimensiones y PriorityScore:

1. As an administrator, I want to schedule maintenance se

In [31]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-025
project = {
    "name": "Project Monitoring System",
    "Description": "A system that allows users to book sports courts, view availability, and manage reservation cancellations.",
    "guidelines": [
        "Personal data security",
        "Plan personalization",
        "Meal reminders",
        "Recipe access",
        "Intuitive interface"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to log my daily meals to keep track of my diet."},
        {"id": 2, "description": "As a user, I want to receive reminders for each meal."},
        {"id": 3, "description": "As an administrator, I want usage reports for the application."},
        {"id": 4, "description": "As a user, I want to receive healthy recipe suggestions based on my preferences."},
        {"id": 5, "description": "As a user, I want to set nutrient intake goals."},
        {"id": 6, "description": "As an administrator, I want to see statistics on the most logged foods."},
        {"id": 7, "description": "As a user, I want access to a list of foods allowed based on my diet."},
        {"id": 8, "description": "As a user, I want to receive alerts about the intake of certain nutrients."},
        {"id": 9, "description": "As an administrator, I want to manage access permissions for advanced features."},
        {"id": 10, "description": "As a user, I want to view charts of my nutritional progress."},
        {"id": 11, "description": "As a user, I want to receive recommendations from other users with similar preferences."},
        {"id": 12, "description": "As an administrator, I want app customization options by region."},
        {"id": 13, "description": "As a user, I want access to a history of all my logged meals."},
        {"id": 14, "description": "As a user, I want to mark recipes as favorites."},
        {"id": 15, "description": "As an administrator, I want user satisfaction reports."},
        {"id": 16, "description": "As a user, I want to receive motivational messages about eating habits."},
        {"id": 17, "description": "As a user, I want to customize my health goals within the app."},
        {"id": 18, "description": "As an administrator, I want to see healthy eating trends on the platform."},
        {"id": 19, "description": "As a user, I want to sync my progress with my device’s calendar."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Project Monitoring System
Guidelines: ['Personal data security', 'Plan personalization', 'Meal reminders', 'Recipe access', 'Intuitive interface']
Dimensiones asociadas (7): ['Risks', 'System-related Benefits', 'Technical Context', 'Benefits', 'Market-related Benefits', 'Project-related Benefits', 'Stakeholder Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  228: Number of use cases inherited by this use case
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  2: Linkage to overall system goals
  31: Importance to release goal
  37: Relevance to stakeholders' goals
  33: Feature contribution to project vision
  30: Relevance to project success
  236: Dependencies due to downstream activities
  197: Experience on the area
  3: 

In [32]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-026
project = {
    "name": "Customer Account Management System",
    "Description": "A system that enables manufacturing companies to monitor and control product quality in real time, reducing defects.",
    "guidelines": [
        "Scalability for large groups",
        "Data security",
        "Content personalization",
        "Progress reports",
        "Cross-platform access"
    ],
    "requirements": [
        {"id": 1, "description": "As an administrator, I want to assign specific courses to each employee."},
        {"id": 2, "description": "As an employee, I want to view my progress in each course."},
        {"id": 3, "description": "As an instructor, I want to conduct evaluations at the end of each module."},
        {"id": 4, "description": "As an administrator, I want performance reports by team."},
        {"id": 5, "description": "As an employee, I want to receive completion certificates."},
        {"id": 6, "description": "As an instructor, I want to monitor each student’s progress."},
        {"id": 7, "description": "As an administrator, I want to manage access permissions for each course."},
        {"id": 8, "description": "As an employee, I want access to supplementary materials in each course."},
        {"id": 9, "description": "As an administrator, I want statistics on the most popular courses."},
        {"id": 10, "description": "As an employee, I want to participate in discussion forums by topic."},
        {"id": 11, "description": "As an instructor, I want to customize modules based on the team’s needs."},
        {"id": 12, "description": "As an administrator, I want reports on employee satisfaction with the course."},
        {"id": 13, "description": "As an employee, I want to receive notifications about upcoming relevant courses."},
        {"id": 14, "description": "As an instructor, I want access to video conferencing tools on the platform."},
        {"id": 15, "description": "As an administrator, I want to manage promotion campaigns for specific courses."},
        {"id": 16, "description": "As an employee, I want to review a history of all the courses I’ve completed."},
        {"id": 17, "description": "As an administrator, I want to see the impact of training on team performance."},
        {"id": 18, "description": "As an employee, I want to save personal notes during the course."},
        {"id": 19, "description": "As an administrator, I want access to a monthly summary of training activity."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Customer Account Management System
Guidelines: ['Scalability for large groups', 'Data security', 'Content personalization', 'Progress reports', 'Cross-platform access']
Dimensiones asociadas (7): ['Technical Context', 'Risks', 'System-related Benefits', 'Customer Benefits', 'Stakeholder Benefits', 'Operational Benefits', 'Costs']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  2: Linkage to overall system goals
  37: Relevance to stakeholders' goals
  52: Contribution to user task
  236: Dependencies due to downstream activities
  197: Experience on the area
  110: Technical risk in current system
  111: Technical risk in proposed system
  215: Sufficient detail in specification
  105: Impact on essential non-functional requirements
  198: Experience on development methodology
  200: Motivation of the team
  150: Impediment of attaining the requirement
  196: Familiarity of the life-cycle m

In [33]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-026
project = {
    "name": "Customer Account Management System",
    "Description": "A system that enables manufacturing companies to monitor and control product quality in real time, reducing defects.",
    "guidelines": [
        "Scalability for large groups",
        "Data security",
        "Content personalization",
        "Progress reports",
        "Cross-platform access"
    ],
    "requirements": [
        {"id": 1, "description": "As an administrator, I want to assign specific courses to each employee."},
        {"id": 2, "description": "As an employee, I want to view my progress in each course."},
        {"id": 3, "description": "As an instructor, I want to conduct evaluations at the end of each module."},
        {"id": 4, "description": "As an administrator, I want performance reports by team."},
        {"id": 5, "description": "As an employee, I want to receive completion certificates."},
        {"id": 6, "description": "As an instructor, I want to monitor each student’s progress."},
        {"id": 7, "description": "As an administrator, I want to manage access permissions for each course."},
        {"id": 8, "description": "As an employee, I want access to supplementary materials in each course."},
        {"id": 9, "description": "As an administrator, I want statistics on the most popular courses."},
        {"id": 10, "description": "As an employee, I want to participate in discussion forums by topic."},
        {"id": 11, "description": "As an instructor, I want to customize modules based on the team’s needs."},
        {"id": 12, "description": "As an administrator, I want reports on employee satisfaction with the course."},
        {"id": 13, "description": "As an employee, I want to receive notifications about upcoming relevant courses."},
        {"id": 14, "description": "As an instructor, I want access to video conferencing tools on the platform."},
        {"id": 15, "description": "As an administrator, I want to manage promotion campaigns for specific courses."},
        {"id": 16, "description": "As an employee, I want to review a history of all the courses I’ve completed."},
        {"id": 17, "description": "As an administrator, I want to see the impact of training on team performance."},
        {"id": 18, "description": "As an employee, I want to save personal notes during the course."},
        {"id": 19, "description": "As an administrator, I want access to a monthly summary of training activity."}
    ]
}

associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Customer Account Management System
Guidelines: ['Scalability for large groups', 'Data security', 'Content personalization', 'Progress reports', 'Cross-platform access']
Dimensiones asociadas (7): ['Technical Context', 'Risks', 'System-related Benefits', 'Customer Benefits', 'Stakeholder Benefits', 'Operational Benefits', 'Costs']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  2: Linkage to overall system goals
  37: Relevance to stakeholders' goals
  52: Contribution to user task
  236: Dependencies due to downstream activities
  197: Experience on the area
  110: Technical risk in current system
  111: Technical risk in proposed system
  215: Sufficient detail in specification
  105: Impact on essential non-functional requirements
  198: Experience on development methodology
  200: Motivation of the team
  150: Impediment of attaining the requirement
  196: Familiarity of the life-cycle m

In [34]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-027
project = {
    "name": "Nutrition and Health Application",
    "Description": "An app that allows small and medium-sized businesses to track income, expenses, and financial projections, supporting decision-making.",
    "guidelines": [
        "Ease of use",
        "Confirmation notifications",
        "Cancellation management",
        "Secure payments",
        "Scalability for different sports"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to see real-time availability of sports courts."},
        {"id": 2, "description": "As a user, I want to book a court to avoid scheduling conflicts."},
        {"id": 3, "description": "As an administrator, I want to manage the availability of courts."},
        {"id": 4, "description": "As a user, I want to receive a confirmation of my reservation via email and SMS."},
        {"id": 5, "description": "As a user, I want to cancel or modify my reservation online."},
        {"id": 6, "description": "As an administrator, I want to view usage statistics for each court."},
        {"id": 7, "description": "As a user, I want to be able to pay online for the reservation."},
        {"id": 8, "description": "As an administrator, I want occupancy reports by sport type."},
        {"id": 9, "description": "As a user, I want to see photos and information about the sports facilities."},
        {"id": 10, "description": "As an administrator, I want to generate income reports for each court."},
        {"id": 11, "description": "As a user, I want to receive alerts about promotions and discounts."},
        {"id": 12, "description": "As an administrator, I want to manage access to the facilities based on reservation type."},
        {"id": 13, "description": "As a user, I want to rate and comment on the quality of the facilities."},
        {"id": 14, "description": "As an administrator, I want to view a reservation history by customer."},
        {"id": 15, "description": "As a user, I want to receive recommendations for sports or activities based on my preferences."},
        {"id": 16, "description": "As an administrator, I want to manage schedule assignments to maximize facility use."},
        {"id": 17, "description": "As a user, I want equipment rental options along with the reservation."},
        {"id": 18, "description": "As an administrator, I want user satisfaction reports."},
        {"id": 19, "description": "As a user, I want to receive notifications of court availability at my preferred time."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Nutrition and Health Application
Guidelines: ['Ease of use', 'Confirmation notifications', 'Cancellation management', 'Secure payments', 'Scalability for different sports']
Dimensiones asociadas (7): ['Technical Context', 'Risks', 'Business Context', 'System-related Benefits', 'Operational Benefits', 'Market-related Benefits', 'Customer Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  52: Contribution to user task
  228: Number of use cases inherited by this use case
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  172: Time to market
  197: Experience on the area
  236: Dependencies due to downstream activities
  2: Linkage to overall system goals
  183: Integration to external systems
  17

In [35]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-028
project = {
    "name": "Online Legal Consultation Platform",
    "Description": "An app that helps users set and track daily habits to improve productivity and well-being.",
    "guidelines": [
        "Personal data security",
        "Quick client info access",
        "Billing integration",
        "Customer reports",
        "User-friendly interface"
    ],
    "requirements": [
        {"id": 1, "description": "As an administrator, I want to record basic information for each client."},
        {"id": 2, "description": "As a manager, I want to view the transaction history of each client."},
        {"id": 3, "description": "As an administrator, I want to manage access permissions based on roles."},
        {"id": 4, "description": "As a user, I want to be able to update my personal account information."},
        {"id": 5, "description": "As a manager, I want activity reports on frequent clients."},
        {"id": 6, "description": "As an administrator, I want to integrate the client account with billing."},
        {"id": 7, "description": "As a user, I want to receive a monthly summary of my transactions."},
        {"id": 8, "description": "As an administrator, I want to manage the client's contact preferences."},
        {"id": 9, "description": "As a manager, I want to view customer complaints and requests."},
        {"id": 10, "description": "As an administrator, I want options to archive inactive accounts."},
        {"id": 11, "description": "As a user, I want to see a history of my purchases and payments."},
        {"id": 12, "description": "As an administrator, I want client account reports by market segment."},
        {"id": 13, "description": "As a manager, I want to compare activity across different customer segments."},
        {"id": 14, "description": "As an administrator, I want to configure inactivity alerts for clients."},
        {"id": 15, "description": "As a user, I want to be able to add and manage payment methods."},
        {"id": 16, "description": "As a manager, I want access to customer loyalty statistics."},
        {"id": 17, "description": "As an administrator, I want customer satisfaction reports."},
        {"id": 18, "description": "As a manager, I want options to segment the customer database."},
        {"id": 19, "description": "As a user, I want to receive product or service recommendations based on my history."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Online Legal Consultation Platform
Guidelines: ['Personal data security', 'Quick client info access', 'Billing integration', 'Customer reports', 'User-friendly interface']
Dimensiones asociadas (7): ['Market-related Benefits', 'Operational Benefits', 'Technical Context', 'Customer Benefits', 'Costs', 'Risks', 'Penalties']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  160: Detraction from product's value
  52: Contribution to user task
  236: Dependencies due to downstream activities
  197: Experience on the area
  110: Technical risk in current system
  111: Technical risk in proposed system
  159: Penalty in operation
  215: Sufficient detail in specification
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  198: Experience on development methodology
  228: Number of use cases inherited by this use case
  224: Number of use cases this use

In [36]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-029
project = {
    "name": "Contract Management System",
    "Description": "An app that allows users to access deals and discounts at nearby stores, making it easy to find products on promotion.",
    "guidelines": [
        "Personal data security",
        "Efficient vacancy management",
        "Access to detailed profiles",
        "Hiring reports",
        "Social media integration"
    ],
    "requirements": [
        {"id": 1, "description": "As a recruiter, I want to post job openings and detail the requirements for each position."},
        {"id": 2, "description": "As a candidate, I want to create a professional profile to apply for jobs."},
        {"id": 3, "description": "As an administrator, I want reports on active and inactive job listings."},
        {"id": 4, "description": "As a recruiter, I want to filter candidates based on their skills and experience."},
        {"id": 5, "description": "As a candidate, I want to receive notifications about new jobs that match my profile."},
        {"id": 6, "description": "As an administrator, I want options to manage recruiter access."},
        {"id": 7, "description": "As a candidate, I want to upload and update my resume."},
        {"id": 8, "description": "As a recruiter, I want to contact candidates through the platform."},
        {"id": 9, "description": "As an administrator, I want to view statistics on the most applied-to job postings."},
        {"id": 10, "description": "As a candidate, I want to review the status of my job applications."},
        {"id": 11, "description": "As a recruiter, I want to compare candidates to facilitate selection."},
        {"id": 12, "description": "As an administrator, I want reports on average hiring times."},
        {"id": 13, "description": "As a candidate, I want to receive alerts for scheduled interviews."},
        {"id": 14, "description": "As a recruiter, I want to share job openings on social media."},
        {"id": 15, "description": "As a candidate, I want to receive job recommendations based on my profile."},
        {"id": 16, "description": "As an administrator, I want to analyze hiring metrics by department."},
        {"id": 17, "description": "As a candidate, I want to set up job alerts based on my preferences."},
        {"id": 18, "description": "As a recruiter, I want access to a database of qualified candidates."},
        {"id": 19, "description": "As an administrator, I want satisfaction reports from recruiters and candidates."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Contract Management System
Guidelines: ['Personal data security', 'Efficient vacancy management', 'Access to detailed profiles', 'Hiring reports', 'Social media integration']
Dimensiones asociadas (7): ['Project-related Benefits', 'Operational Benefits', 'Risks', 'Business Context', 'System-related Benefits', 'Strategy-related Benefits', 'Stakeholder Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  172: Time to market
  183: Integration to external systems
  2: Linkage to overall system goals
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  37: Relevance to stakeholders' goals
  43: Importance to business goals
  44: Criticality to mission success
  52: Contribution to user task
  77: Control and follow-up
  105: Impact on essential non-functional requirements
  56: Cost reduction of IT operation
  79: Integration and coordination

Requisitos con sus subdimensiones y Prior

In [37]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-030
project = {
    "name": "Sports Field Reservation System",
    "Description": "A platform that allows companies to train their employees online, with progress tracking and assessments.",
    "guidelines": [
        "Data security and confidentiality",
        "Expiration management",
        "Automatic notifications",
        "Role-based access",
        "Activity reports"
    ],
    "requirements": [
        {"id": 1, "description": "As an administrator, I want to securely create and store contracts."},
        {"id": 2, "description": "As a manager, I want to receive notifications before contracts expire."},
        {"id": 3, "description": "As an administrator, I want to assign access permissions to specific contracts."},
        {"id": 4, "description": "As a user, I want to view a change history for each contract."},
        {"id": 5, "description": "As a manager, I want reports on contracts that are about to expire."},
        {"id": 6, "description": "As an administrator, I want to manage the automatic renewal of certain contracts."},
        {"id": 7, "description": "As a user, I want to securely access contracts from any device."},
        {"id": 8, "description": "As an administrator, I want activity reports for each contract."},
        {"id": 9, "description": "As a manager, I want to see the average duration of the company’s contracts."},
        {"id": 10, "description": "As an administrator, I want to securely archive expired contracts."},
        {"id": 11, "description": "As a user, I want to receive alerts for contracts that require review."},
        {"id": 12, "description": "As a manager, I want contract compliance statistics."},
        {"id": 13, "description": "As an administrator, I want to manage access to the contract database by role."},
        {"id": 14, "description": "As a user, I want options to download copies of my contracts."},
        {"id": 15, "description": "As a manager, I want to compare the profitability of similar contracts."},
        {"id": 16, "description": "As an administrator, I want to set alerts for contract breaches."},
        {"id": 17, "description": "As a user, I want to receive renewal recommendations based on market conditions."},
        {"id": 18, "description": "As an administrator, I want a complete history of all signed contracts."},
        {"id": 19, "description": "As a manager, I want reports on contract effectiveness in each business area."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Sports Field Reservation System
Guidelines: ['Data security and confidentiality', 'Expiration management', 'Automatic notifications', 'Role-based access', 'Activity reports']
Dimensiones asociadas (7): ['Operational Benefits', 'Risks', 'Technical Context', 'System-related Benefits', 'Benefits', 'Market-related Benefits', 'Project-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  197: Experience on the area
  52: Contribution to user task
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  198: Experience on development methodology
  2: Linkage to overall system goals
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  236: Dependencies due to downstream activities
  225: Number of use cases that includes this use case
  227: Number of use cases that extend this use

In [38]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-031
project = {
    "name": "Vehicle Maintenance Management System",
    "Description": "A platform that allows companies to manage employee information, including contracts, benefits, and performance.",
    "guidelines": [
        "Real-time access",
        "Progress notifications",
        "Performance reports",
        "Tool integration",
        "View personalization"
    ],
    "requirements": [
        {"id": 1, "description": "As a manager, I want to see the status of each project in real time."},
        {"id": 2, "description": "As an administrator, I want to assign view permissions based on roles."},
        {"id": 3, "description": "As a user, I want to update the progress of my assigned tasks."},
        {"id": 4, "description": "As a manager, I want to receive alerts for delayed projects."},
        {"id": 5, "description": "As an administrator, I want reports on time spent on each project."},
        {"id": 6, "description": "As a user, I want to see a list of all my pending tasks."},
        {"id": 7, "description": "As a manager, I want to view progress charts by project."},
        {"id": 8, "description": "As an administrator, I want to manage integration with other management tools."},
        {"id": 9, "description": "As a manager, I want to compare progress across different projects."},
        {"id": 10, "description": "As a user, I want to receive reminders for upcoming tasks."},
        {"id": 11, "description": "As an administrator, I want to see team performance statistics."},
        {"id": 12, "description": "As a manager, I want access to a summary of each completed project."},
        {"id": 13, "description": "As a user, I want collaboration options with other project members."},
        {"id": 14, "description": "As an administrator, I want reports of completed and pending tasks."},
        {"id": 15, "description": "As a manager, I want to see a cost breakdown for each project."},
        {"id": 16, "description": "As an administrator, I want to define performance goals on the platform."},
        {"id": 17, "description": "As a user, I want to receive notifications of changes in the project."},
        {"id": 18, "description": "As a manager, I want an analysis of each project's effectiveness."},
        {"id": 19, "description": "As an administrator, I want to export project performance reports."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Vehicle Maintenance Management System
Guidelines: ['Real-time access', 'Progress notifications', 'Performance reports', 'Tool integration', 'View personalization']
Dimensiones asociadas (7): ['Business Context', 'Costs', 'Risks', 'System-related Benefits', 'Operational Benefits', 'Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  52: Contribution to user task
  172: Time to market
  183: Integration to external systems
  2: Linkage to overall system goals
  110: Technical risk in current system
  111: Technical risk in proposed system
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  150: Impediment of attaining the requirement
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  3: Ease of use
  188: Frequency of use
  148: Project duration

Requisitos con sus subdimensiones y PriorityScore:

1. As a manager, I want to see the status of each proje

In [39]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-032
project = {
    "name": "Telework Platform",
    "Description": "A platform that allows companies to manage remote work teams, including video conferencing, task assignment, and progress tracking.",
    "guidelines": [
        "Secure communication",
        "Access to collaboration tools",
        "Calendar integration",
        "Productivity reports",
        "User-friendly interface"
    ],
    "requirements": [
        {"id": 1, "description": "As a manager, I want to assign tasks to remote team members."},
        {"id": 2, "description": "As an employee, I want to see a list of assigned tasks and their status."},
        {"id": 3, "description": "As an administrator, I want to manage access permissions for the tools."},
        {"id": 4, "description": "As a manager, I want to access work time statistics for each team member."},
        {"id": 5, "description": "As an employee, I want to participate in video conferences from the platform."},
        {"id": 6, "description": "As an administrator, I want to integrate the platform with calendar tools."},
        {"id": 7, "description": "As a manager, I want to receive productivity reports by project."},
        {"id": 8, "description": "As an employee, I want to mark tasks as completed."},
        {"id": 9, "description": "As an administrator, I want options to store video conference recordings."},
        {"id": 10, "description": "As a manager, I want to visualize each team member’s weekly progress."},
        {"id": 11, "description": "As an employee, I want to receive notifications for tasks nearing their deadlines."},
        {"id": 12, "description": "As an administrator, I want to manage communication security on the platform."},
        {"id": 13, "description": "As a manager, I want to receive alerts for overdue tasks."},
        {"id": 14, "description": "As an employee, I want to access shared resources during meetings."},
        {"id": 15, "description": "As an administrator, I want platform usage reports by company area."},
        {"id": 16, "description": "As a manager, I want to schedule video conferences for regular meetings."},
        {"id": 17, "description": "As an employee, I want to receive reminders for scheduled meetings."},
        {"id": 18, "description": "As an administrator, I want to control guest access to video conferences."},
        {"id": 19, "description": "As a manager, I want a monthly summary of remote team productivity."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Telework Platform
Guidelines: ['Secure communication', 'Access to collaboration tools', 'Calendar integration', 'Productivity reports', 'User-friendly interface']
Dimensiones asociadas (7): ['Operational Benefits', 'Technical Context', 'Project-related Benefits', 'Business Context', 'Stakeholder Benefits', 'Strategy-related Benefits', 'Costs']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  52: Contribution to user task
  30: Relevance to project success
  33: Feature contribution to project vision
  44: Criticality to mission success
  172: Time to market
  183: Integration to external systems
  236: Dependencies due to downstream activities
  31: Importance to release goal
  37: Relevance to stakeholders' goals
  43: Importance to business goals
  197: Experience on the area
  215: Sufficient detail in specification

Requisitos con sus subdimensiones y Pri

In [40]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-033
project = {
    "name": "School Transport Reservation Management System",
    "Description": "A platform to organize and manage events, allowing users to register attendees and coordinate activities for each event.",
    "guidelines": [
        "Financial data security",
        "Income and expense reports",
        "Cross-platform access",
        "Category personalization",
        "Bank integration"
    ],
    "requirements": [
        {"id": 1, "description": "As a business owner, I want to record my income and expenses for accurate tracking."},
        {"id": 2, "description": "As an administrator, I want monthly income and expense reports."},
        {"id": 3, "description": "As a business owner, I want to receive alerts for expenses outside the budget."},
        {"id": 4, "description": "As an administrator, I want to customize income and expense categories."},
        {"id": 5, "description": "As a business owner, I want to see real-time cash flow charts."},
        {"id": 6, "description": "As an administrator, I want expense statistics by business area."},
        {"id": 7, "description": "As a business owner, I want to schedule recurring payments from the app."},
        {"id": 8, "description": "As an administrator, I want to manage access permissions to the application."},
        {"id": 9, "description": "As a business owner, I want to receive invoice due date reminders."},
        {"id": 10, "description": "As an administrator, I want a history of all company transactions."},
        {"id": 11, "description": "As a business owner, I want to view financial projections based on my current income."},
        {"id": 12, "description": "As an administrator, I want to generate exportable reports by category."},
        {"id": 13, "description": "As a business owner, I want to sync my bank accounts to automatically update income and expenses."},
        {"id": 14, "description": "As an administrator, I want profitability reports by project."},
        {"id": 15, "description": "As a business owner, I want to receive investment recommendations based on my cash flow."},
        {"id": 16, "description": "As an administrator, I want access to the company’s financial health metrics."},
        {"id": 17, "description": "As a business owner, I want to manage company taxes through the app."},
        {"id": 18, "description": "As an administrator, I want user satisfaction reports."},
        {"id": 19, "description": "As a business owner, I want to receive alerts for upcoming payments and forecasted expenses."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: School Transport Reservation Management System
Guidelines: ['Financial data security', 'Income and expense reports', 'Cross-platform access', 'Category personalization', 'Bank integration']
Dimensiones asociadas (7): ['Operational Benefits', 'Risks', 'System-related Benefits', 'Costs', 'Penalties', 'Business Context', 'Technical Context']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  160: Detraction from product's value
  110: Technical risk in current system
  111: Technical risk in proposed system
  159: Penalty in operation
  215: Sufficient detail in specification
  52: Contribution to user task
  172: Time to market
  183: Integration to external systems
  236: Dependencies due to downstream activities
  2: Linkage to overall system goals
  197: Experience on the area
  51: Fit with business processes

Requisitos con sus subdimensiones y PriorityScore

In [41]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-034
project = {
    "name": "Online Course Management Platform",
    "Description": "A platform that enables instructors and students to access online courses, manage classes, and perform assessments.",
    "guidelines": [
        "Secure content Access",
        "Updated materials",
        "Scalability for large audiences",
        "Usage reports",
        "User interaction"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to access health courses adapted to my level of knowledge."},
        {"id": 2, "description": "As an administrator, I want to manage access permissions to sensitive content."},
        {"id": 3, "description": "As a user, I want to see a history of all completed courses."},
        {"id": 4, "description": "As an administrator, I want statistics on the most popular courses."},
        {"id": 5, "description": "As a user, I want to participate in discussion forums with other students and professionals."},
        {"id": 6, "description": "As an administrator, I want reports on the effectiveness of each course."},
        {"id": 7, "description": "As a user, I want to receive certificates upon course completion."},
        {"id": 8, "description": "As an administrator, I want to manage material updates based on changes in health regulations."},
        {"id": 9, "description": "As a user, I want to receive reminders about courses in progress."},
        {"id": 10, "description": "As an administrator, I want to view user participation statistics."},
        {"id": 11, "description": "As a user, I want to receive course recommendations based on my interests."},
        {"id": 12, "description": "As an administrator, I want options to customize the educational interface."},
        {"id": 13, "description": "As a user, I want to access multimedia content on health topics."},
        {"id": 14, "description": "As an administrator, I want student satisfaction reports."},
        {"id": 15, "description": "As a user, I want to take assessments to measure my learning."},
        {"id": 16, "description": "As an administrator, I want to manage course feedback surveys."},
        {"id": 17, "description": "As a user, I want to download supplementary materials to study offline."},
        {"id": 18, "description": "As an administrator, I want to view usage trends by user type."},
        {"id": 19, "description": "As a user, I want to receive alerts about new courses related to my profile."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Online Course Management Platform
Guidelines: ['Secure content Access', 'Updated materials', 'Scalability for large audiences', 'Usage reports', 'User interaction']
Dimensiones asociadas (7): ['Technical Context', 'Operational Benefits', 'Stakeholder Benefits', 'Customer Benefits', 'Costs', 'System-related Benefits', 'Risks']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  52: Contribution to user task
  2: Linkage to overall system goals
  37: Relevance to stakeholders' goals
  228: Number of use cases inherited by this use case
  236: Dependencies due to downstream activities
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  197: Experience on the area
  105: Impact on essential non-functional requirements
  198: Experience on devel

In [42]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-035
project = {
    "name": "Sports Training Platform",
    "Description": "An app that helps users improve their health through personalized nutrition plans, food logging, and health tips.",
    "guidelines": [
        "Secure communication",
        "Schedule change notifications",
        "Real-time location access",
        "Usage reports",
        "Scalability for multiple routes"
    ],
    "requirements": [
        {"id": 1, "description": "As a parent, I want to book school transportation for my child."},
        {"id": 2, "description": "As a driver, I want to see the list of students for each assigned route."},
        {"id": 3, "description": "As an administrator, I want to manage driver routes in real time."},
        {"id": 4, "description": "As a parent, I want to receive notifications of the bus arrival and departure."},
        {"id": 5, "description": "As a driver, I want to log arrival and departure times at each stop."},
        {"id": 6, "description": "As an administrator, I want reports on usage and effectiveness of each route."},
        {"id": 7, "description": "As a parent, I want to access the real-time location of the bus."},
        {"id": 8, "description": "As a driver, I want to receive alerts about changes in the assigned routes."},
        {"id": 9, "description": "As an administrator, I want to see usage statistics for each route."},
        {"id": 10, "description": "As a parent, I want to receive delay alerts for the service."},
        {"id": 11, "description": "As an administrator, I want to manage access permissions to the platform."},
        {"id": 12, "description": "As a driver, I want options to mark student attendance."},
        {"id": 13, "description": "As a parent, I want to access a history of transportation bookings for my child."},
        {"id": 14, "description": "As an administrator, I want reports on parent satisfaction with the service."},
        {"id": 15, "description": "As a driver, I want to receive vehicle maintenance alerts."},
        {"id": 16, "description": "As a parent, I want to personalize my alerts according to my needs."},
        {"id": 17, "description": "As an administrator, I want to control the number of students per route."},
        {"id": 18, "description": "As a parent, I want to rate the punctuality and safety of the service."},
        {"id": 19, "description": "As an administrator, I want to see a weekly summary of reported incidents."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Sports Training Platform
Guidelines: ['Secure communication', 'Schedule change notifications', 'Real-time location access', 'Usage reports', 'Scalability for multiple routes']
Dimensiones asociadas (7): ['Technical Context', 'Operational Benefits', 'Business Context', 'Customer Benefits', 'Costs', 'Risks', 'System-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  172: Time to market
  2: Linkage to overall system goals
  52: Contribution to user task
  183: Integration to external systems
  236: Dependencies due to downstream activities
  197: Experience on the area
  150: Impediment of attaining the requirement
  196: Familiarity of the life-cycle model
  200: Motivation of the team
  174: Impact on Maintenance
  105: Impact on essential non-functional requirements
  177: Impact on long-term Evolution

Requisitos con sus subdimensiones y Pr

In [43]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-036
project = {
    "name": "Customer Service System for Companies",
    "Description": "A system that enables companies to manage customer inquiries and requests, providing fast responses and case tracking.",
    "guidelines": [
        "Secure payments",
        "Ease of reservation",
        "Option personalization",
        "Confirmation notifications",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to search for available vehicles to rent according to my preferences."},
        {"id": 2, "description": "As a user, I want to book a vehicle quickly and securely."},
        {"id": 3, "description": "As an administrator, I want to manage vehicle availability in real time."},
        {"id": 4, "description": "As a user, I want to receive reservation confirmations by email and SMS."},
        {"id": 5, "description": "As a user, I want to modify or cancel my reservation online."},
        {"id": 6, "description": "As an administrator, I want usage reports for each type of vehicle."},
        {"id": 7, "description": "As a user, I want to choose insurance options for the rental."},
        {"id": 8, "description": "As an administrator, I want to manage seasonal rates and promotions."},
        {"id": 9, "description": "As a user, I want to receive alerts for discounts or promotions."},
        {"id": 10, "description": "As an administrator, I want statistics on the most rented vehicles."},
        {"id": 11, "description": "As a user, I want to rate the quality of the vehicle after use."},
        {"id": 12, "description": "As an administrator, I want to control access to certain vehicle categories."},
        {"id": 13, "description": "As a user, I want rental options for additional equipment like GPS or child seats."},
        {"id": 14, "description": "As an administrator, I want to manage vehicle requests in different cities."},
        {"id": 15, "description": "As a user, I want to receive renewal notifications if my rental is extended."},
        {"id": 16, "description": "As an administrator, I want income and expense reports for the service."},
        {"id": 17, "description": "As a user, I want to review a history of my previous rentals."},
        {"id": 18, "description": "As an administrator, I want options to promote new vehicle models."},
        {"id": 19, "description": "As a user, I want payment options in different currencies."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Customer Service System for Companies
Guidelines: ['Secure payments', 'Ease of reservation', 'Option personalization', 'Confirmation notifications', 'Scalability']
Dimensiones asociadas (7): ['Risks', 'Technical Context', 'System-related Benefits', 'Business Context', 'Operational Benefits', 'Market-related Benefits', 'Financial Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  52: Contribution to user task
  227: Number of use cases that extend this use case
  228: Number of use cases inherited by this use case
  197: Experience on the area
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  172: Time to market
  2: Linkage to overall system goals
  183: Integration to external systems
  236: Dependencies due to downstream activities
  26: Featur

In [44]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-037
project = {
    "name": "Inventory Management System for Restaurants",
    "Description": "A system for managing and monitoring project progress in real time, enabling managers to make data-driven decisions.",
    "guidelines": [
        "Financial data security",
        "Landlord-tenant communication",
        "Payment reports",
        "Due date notifications",
        "Scalability for multiple properties"
    ],
    "requirements": [
        {"id": 1, "description": "As a landlord, I want to manage my property information on a platform."},
        {"id": 2, "description": "As a tenant, I want to receive notifications about payment due dates."},
        {"id": 3, "description": "As a landlord, I want to view a payment history for each tenant."},
        {"id": 4, "description": "As a tenant, I want to make secure payments through the platform."},
        {"id": 5, "description": "As a landlord, I want monthly and annual income reports for each property."},
        {"id": 6, "description": "As an administrator, I want to manage access permissions to property information."},
        {"id": 7, "description": "As a landlord, I want options to contact tenants directly from the platform."},
        {"id": 8, "description": "As a tenant, I want to review my rental agreement in the app."},
        {"id": 9, "description": "As a landlord, I want to receive alerts for maintenance requests from tenants."},
        {"id": 10, "description": "As an administrator, I want reports on occupancy and vacancies of properties."},
        {"id": 11, "description": "As a landlord, I want to view the request history for each tenant."},
        {"id": 12, "description": "As a tenant, I want to rate the quality of service provided by the landlord."},
        {"id": 13, "description": "As an administrator, I want to manage rental rates flexibly."},
        {"id": 14, "description": "As a tenant, I want to receive reminders for pending payments."},
        {"id": 15, "description": "As a landlord, I want performance statistics for each property."},
        {"id": 16, "description": "As a tenant, I want options to renew my lease through the platform."},
        {"id": 17, "description": "As an administrator, I want to export financial reports for each property."},
        {"id": 18, "description": "As a tenant, I want options to submit maintenance requests."},
        {"id": 19, "description": "As a landlord, I want to receive tenant satisfaction reports."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Inventory Management System for Restaurants
Guidelines: ['Financial data security', 'Landlord-tenant communication', 'Payment reports', 'Due date notifications', 'Scalability for multiple properties']
Dimensiones asociadas (7): ['Technical Context', 'System-related Benefits', 'Customer Benefits', 'Operational Benefits', 'Risks', 'Penalties', 'Business Context']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  160: Detraction from product's value
  236: Dependencies due to downstream activities
  2: Linkage to overall system goals
  52: Contribution to user task
  172: Time to market
  183: Integration to external systems
  197: Experience on the area
  53: Support for main usage scenario
  174: Impact on Maintenance
  47: Efficiency gains for customer
  48: Competitive gains for customer
  162: Negative value for customer

Requisitos con sus subdimensiones y 

In [45]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-038
project = {
    "name": "Health Education Platform",
    "Description": "A platform that connects students with qualified tutors in various subjects for personalized academic support.",
    "guidelines": [
        "Secure content access",
        "Scalability for large groups",
        "Progress reports",
        "Course personalization",
        "Video integration"
    ],
    "requirements": [
        {"id": 1, "description": "As an instructor, I want to create courses and organize the content into modules."},
        {"id": 2, "description": "As a student, I want to enroll in courses that interest me."},
        {"id": 3, "description": "As an administrator, I want to manage course access permissions based on roles."},
        {"id": 4, "description": "As an instructor, I want to conduct evaluations of students at the end of each module."},
        {"id": 5, "description": "As a student, I want to receive certificates upon completing a course."},
        {"id": 6, "description": "As an administrator, I want performance reports on the courses."},
        {"id": 7, "description": "As an instructor, I want to upload videos and support materials for the course."},
        {"id": 8, "description": "As a student, I want to view a history of completed courses."},
        {"id": 9, "description": "As an administrator, I want statistics on the most popular courses."},
        {"id": 10, "description": "As an instructor, I want to access discussion forums with students."},
        {"id": 11, "description": "As a student, I want to receive reminders for upcoming classes or evaluations."},
        {"id": 12, "description": "As an administrator, I want options to customize the platform interface."},
        {"id": 13, "description": "As an instructor, I want access to video conferencing tools on the platform."},
        {"id": 14, "description": "As a student, I want to complete self-assessment quizzes."},
        {"id": 15, "description": "As an administrator, I want to manage promotional campaigns for certain courses."},
        {"id": 16, "description": "As a student, I want to receive recommendations for related courses."},
        {"id": 17, "description": "As an instructor, I want to receive feedback from students."},
        {"id": 18, "description": "As an administrator, I want reports on student satisfaction."},
        {"id": 19, "description": "As a student, I want access to a frequently asked questions section about the course."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Health Education Platform
Guidelines: ['Secure content access', 'Scalability for large groups', 'Progress reports', 'Course personalization', 'Video integration']
Dimensiones asociadas (7): ['Technical Context', 'Customer Benefits', 'Operational Benefits', 'Costs', 'Business Context', 'System-related Benefits', 'Stakeholder Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  184: System value of a feature
  183: Integration to external systems
  2: Linkage to overall system goals
  37: Relevance to stakeholders' goals
  52: Contribution to user task
  172: Time to market
  236: Dependencies due to downstream activities
  197: Experience on the area
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  198: Experience on development methodology
  215: Sufficient detail in specification
  196: Familiarity of the life-cycle model

Requisitos con sus subdimensiones y Priori

In [46]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-039
project = {
    "name": "Property Management Platform",
    "Description": "A platform for property owners to manage rentals, make payments, and communicate with tenants.",
    "guidelines": [
        "Cross-platform access",
        "Secure payments",
        "Offer notifications",
        "Usage reports",
        "Map integration"
    ],
    "requirements": [
        {"id": 1, "description": "As a user, I want to see discounts and promotions in nearby stores."},
        {"id": 2, "description": "As a merchant, I want to publish product offers in real time."},
        {"id": 3, "description": "As an administrator, I want reports on the most popular product categories."},
        {"id": 4, "description": "As a user, I want to receive offer notifications based on my preferences."},
        {"id": 5, "description": "As a merchant, I want to manage the availability of products on promotion."},
        {"id": 6, "description": "As an administrator, I want usage statistics of the application."},
        {"id": 7, "description": "As a user, I want to access a map showing the location of stores with promotions."},
        {"id": 8, "description": "As a merchant, I want promotion options in the app based on the season."},
        {"id": 9, "description": "As a user, I want to rate the quality of store products and services."},
        {"id": 10, "description": "As an administrator, I want user satisfaction reports."},
        {"id": 11, "description": "As a user, I want to be able to share promotions on social media."},
        {"id": 12, "description": "As a merchant, I want to manage personalized promotions for frequent customers."},
        {"id": 13, "description": "As a user, I want access to a history of promotions I’ve participated in."},
        {"id": 14, "description": "As an administrator, I want to manage marketing campaigns within the app."},
        {"id": 15, "description": "As a user, I want to receive alerts for out-of-stock products."},
        {"id": 16, "description": "As a merchant, I want to analyze the impact of promotions on my sales."},
        {"id": 17, "description": "As a user, I want to see product recommendations based on my previous purchases."},
        {"id": 18, "description": "As a merchant, I want to offer exclusive discounts to app users."},
        {"id": 19, "description": "As a user, I want online payment options for in-store promotions."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Property Management Platform
Guidelines: ['Cross-platform access', 'Secure payments', 'Offer notifications', 'Usage reports', 'Map integration']
Dimensiones asociadas (7): ['Operational Benefits', 'Technical Context', 'Costs', 'Business Context', 'Benefits', 'System-related Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  52: Contribution to user task
  197: Experience on the area
  228: Number of use cases inherited by this use case
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  172: Time to market
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  26: Feature influence on buying decision
  183: Integration to external systems

Requisitos con su

In [47]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-040
project = {
    "name": "Inventory Management System for Retailers",
    "Description": "A system that allows companies to manage the maintenance of their vehicle fleets, scheduling services and recording maintenance history.",
    "guidelines": [
        "Real-time inventory control",
        "Low stock notifications",
        "Supplier integration",
        "Sales reports",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As a retailer, I want to see my inventory in real time to control my stock levels."},
        {"id": 2, "description": "As an administrator, I want to receive alerts when inventory is low."},
        {"id": 3, "description": "As a retailer, I want to place replenishment orders automatically."},
        {"id": 4, "description": "As an administrator, I want to see sales statistics by product category."},
        {"id": 5, "description": "As a retailer, I want to adjust prices and promotions flexibly."},
        {"id": 6, "description": "As an administrator, I want to manage access permissions according to roles in the system."},
        {"id": 7, "description": "As a retailer, I want to access a sales history to forecast demand."},
        {"id": 8, "description": "As an administrator, I want to manage relationships with suppliers."},
        {"id": 9, "description": "As a retailer, I want to see sales charts and product trends."},
        {"id": 10, "description": "As an administrator, I want profitability reports for each product."},
        {"id": 11, "description": "As a retailer, I want to integrate the platform with my point-of-sale system."},
        {"id": 12, "description": "As an administrator, I want statistics on best-selling products by region."},
        {"id": 13, "description": "As a retailer, I want custom notification options for stock management."},
        {"id": 14, "description": "As an administrator, I want reports of losses due to damaged or expired products."},
        {"id": 15, "description": "As a retailer, I want to manage product availability across different stores."},
        {"id": 16, "description": "As an administrator, I want to compare the effectiveness of promotions across products."},
        {"id": 17, "description": "As a retailer, I want order recommendations based on demand."},
        {"id": 18, "description": "As an administrator, I want to centrally manage inventory and pricing."},
        {"id": 19, "description": "As a retailer, I want access to inventory planning tools for special events."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Inventory Management System for Retailers
Guidelines: ['Real-time inventory control', 'Low stock notifications', 'Supplier integration', 'Sales reports', 'Scalability']
Dimensiones asociadas (7): ['Operational Benefits', 'Business Context', 'Costs', 'Risks', 'System-related Benefits', 'Market-related Benefits', 'Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  2: Linkage to overall system goals
  172: Time to market
  183: Integration to external systems
  52: Contribution to user task
  110: Technical risk in current system
  111: Technical risk in proposed system
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  175: Importance for product architecture
  56: Cost reduction of IT operation
  77: Control and follow-up
  15: Product Market value

Requisitos con sus subdimensiones y PriorityScore:

1. As a retailer, I want to

In [48]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-040
project = {
    "name": "Inventory Management System for Retailers",
    "Description": "A system that allows companies to manage the maintenance of their vehicle fleets, scheduling services and recording maintenance history.",
    "guidelines": [
        "Real-time inventory control",
        "Low stock notifications",
        "Supplier integration",
        "Sales reports",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As a retailer, I want to see my inventory in real time to control my stock levels."},
        {"id": 2, "description": "As an administrator, I want to receive alerts when inventory is low."},
        {"id": 3, "description": "As a retailer, I want to place replenishment orders automatically."},
        {"id": 4, "description": "As an administrator, I want to see sales statistics by product category."},
        {"id": 5, "description": "As a retailer, I want to adjust prices and promotions flexibly."},
        {"id": 6, "description": "As an administrator, I want to manage access permissions according to roles in the system."},
        {"id": 7, "description": "As a retailer, I want to access a sales history to forecast demand."},
        {"id": 8, "description": "As an administrator, I want to manage relationships with suppliers."},
        {"id": 9, "description": "As a retailer, I want to see sales charts and product trends."},
        {"id": 10, "description": "As an administrator, I want profitability reports for each product."},
        {"id": 11, "description": "As a retailer, I want to integrate the platform with my point-of-sale system."},
        {"id": 12, "description": "As an administrator, I want statistics on best-selling products by region."},
        {"id": 13, "description": "As a retailer, I want custom notification options for stock management."},
        {"id": 14, "description": "As an administrator, I want reports of losses due to damaged or expired products."},
        {"id": 15, "description": "As a retailer, I want to manage product availability across different stores."},
        {"id": 16, "description": "As an administrator, I want to compare the effectiveness of promotions across products."},
        {"id": 17, "description": "As a retailer, I want order recommendations based on demand."},
        {"id": 18, "description": "As an administrator, I want to centrally manage inventory and pricing."},
        {"id": 19, "description": "As a retailer, I want access to inventory planning tools for special events."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Inventory Management System for Retailers
Guidelines: ['Real-time inventory control', 'Low stock notifications', 'Supplier integration', 'Sales reports', 'Scalability']
Dimensiones asociadas (7): ['Operational Benefits', 'Business Context', 'Costs', 'Risks', 'System-related Benefits', 'Market-related Benefits', 'Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  2: Linkage to overall system goals
  172: Time to market
  183: Integration to external systems
  52: Contribution to user task
  110: Technical risk in current system
  111: Technical risk in proposed system
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  175: Importance for product architecture
  56: Cost reduction of IT operation
  77: Control and follow-up
  15: Product Market value

Requisitos con sus subdimensiones y PriorityScore:

1. As a retailer, I want to

In [49]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-041
project = {
    "name": "Menstrual Cycle Tracking Application",
    "Description": "An app that allows users to record and track their menstrual cycle, offering predictions and well-being recommendations.",
    "guidelines": [
        "Training plan personalization",
        "Cross-platform access",
        "Personal data security",
        "Progress reports",
        "Wearable integration"
    ],
    "requirements": [
        {"id": 1, "description": "As a trainer, I want to create personalized training plans for my clients."},
        {"id": 2, "description": "As a client, I want to view my training plan and log my daily activities."},
        {"id": 3, "description": "As an administrator, I want to manage access permissions to personal data."},
        {"id": 4, "description": "As a trainer, I want to monitor my clients’ progress in real time."},
        {"id": 5, "description": "As a client, I want to receive reminders for my training sessions."},
        {"id": 6, "description": "As an administrator, I want user satisfaction reports."},
        {"id": 7, "description": "As a client, I want to see charts of my physical and health progress."},
        {"id": 8, "description": "As a trainer, I want to access my clients’ performance statistics."},
        {"id": 9, "description": "As a client, I want to sync my activity with wearable devices."},
        {"id": 10, "description": "As an administrator, I want interface customization options."},
        {"id": 11, "description": "As a trainer, I want to receive alerts about client progress or inactivity."},
        {"id": 12, "description": "As a client, I want access to a history of my previous workouts."},
        {"id": 13, "description": "As a trainer, I want to suggest nutrition recommendations to my clients."},
        {"id": 14, "description": "As an administrator, I want reports on platform usage trends."},
        {"id": 15, "description": "As a client, I want to mark certain exercises as favorites to review later."},
        {"id": 16, "description": "As a trainer, I want to share educational content about health and sports."},
        {"id": 17, "description": "As an administrator, I want reports on the effectiveness of training programs."},
        {"id": 18, "description": "As a client, I want to receive notifications about new exercises and techniques."},
        {"id": 19, "description": "As a trainer, I want to schedule evaluation sessions with clients."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Menstrual Cycle Tracking Application
Guidelines: ['Training plan personalization', 'Cross-platform access', 'Personal data security', 'Progress reports', 'Wearable integration']
Dimensiones asociadas (7): ['Risks', 'System-related Benefits', 'Operational Benefits', 'Costs', 'Business Context', 'Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  110: Technical risk in current system
  111: Technical risk in proposed system
  52: Contribution to user task
  172: Time to market
  183: Integration to external systems
  2: Linkage to overall system goals
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  56: Cost reduction of IT operation
  186: IT departments technical guidelines
  53: Support for main usage scenario
  79: Integration and coordination

Requisitos con sus subdimensiones y PriorityScore:



In [50]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-042
project = {
    "name": "Quality Control System for Manufacturing",
    "Description": "A system for businesses that facilitates contract creation, tracking, secure storage, and management of expirations and renewals.",
    "guidelines": [
        "Medical information security",
        "Automated reminders",
        "Medical system integration",
        "Ease of use",
        "Cross-platform access"
    ],
    "requirements": [
        {"id": 1, "description": "As a patient, I want to book medical appointments online to avoid phone calls."},
        {"id": 2, "description": "As a doctor, I want to view a brief medical history of the patient before the consultation."},
        {"id": 3, "description": "As an administrator, I want reports of completed and cancelled consultations."},
        {"id": 4, "description": "As a patient, I want to receive reminders of my appointments via SMS and email."},
        {"id": 5, "description": "As an administrator, I want to manage access permissions based on roles in the clinic."},
        {"id": 6, "description": "As a doctor, I want options to record notes from each consultation."},
        {"id": 7, "description": "As a patient, I want access to my previous consultation history."},
        {"id": 8, "description": "As an administrator, I want to see statistics on doctors’ availability."},
        {"id": 9, "description": "As a patient, I want to receive follow-up or test recommendations after a consultation."},
        {"id": 10, "description": "As a doctor, I want to schedule follow-up appointments based on each patient's condition."},
        {"id": 11, "description": "As an administrator, I want reports on treatment effectiveness in the clinic."},
        {"id": 12, "description": "As a patient, I want options to cancel or reschedule my appointment online."},
        {"id": 13, "description": "As a doctor, I want to receive alerts for upcoming appointments and schedule changes."},
        {"id": 14, "description": "As a patient, I want access to a list of my scheduled appointments."},
        {"id": 15, "description": "As an administrator, I want to manage communication between patients and doctors on the platform."},
        {"id": 16, "description": "As a patient, I want options to share my medical history with other doctors."},
        {"id": 17, "description": "As an administrator, I want integration options with billing systems."},
        {"id": 18, "description": "As a doctor, I want to view my patients’ treatment histories."},
        {"id": 19, "description": "As a patient, I want to receive alerts when appointments become available for quick rescheduling."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Quality Control System for Manufacturing
Guidelines: ['Medical information security', 'Automated reminders', 'Medical system integration', 'Ease of use', 'Cross-platform access']
Dimensiones asociadas (7): ['Risks', 'Technical Context', 'Business Context', 'System-related Benefits', 'Operational Benefits', 'Market-related Benefits', 'Financial Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  183: Integration to external systems
  2: Linkage to overall system goals
  52: Contribution to user task
  172: Time to market
  197: Experience on the area
  236: Dependencies due to downstream activities
  26: Feature influence on buying decision
  39: Return on investment
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  198: Experience on development methodology

Requisitos con sus su

In [51]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-043
project = {
    "name": "Finance Management App for SMEs",
    "Description": "A crowdfunding platform focused on educational projects, allowing users to donate to fund education programs.",
    "guidelines": [
        "Personal data privacy and security",
        "User-friendly interface",
        "Personalized reminders",
        "Well-being reports",
        "Cross-platform access"
    ],
    "requirements": [
        {"id": 1, "description": "As a female user, I want to track my menstrual cycles for accurate monitoring."},
        {"id": 2, "description": "As a female user, I want to receive personalized reminders based on my cycle."},
        {"id": 3, "description": "As a female administrator, I want reports on app usage by region."},
        {"id": 4, "description": "As a female user, I want wellness recommendations based on the phase of my cycle."},
        {"id": 5, "description": "As a female administrator, I want to manage the privacy and security of recorded data."},
        {"id": 6, "description": "As a female user, I want to view graphs of my cycle and its patterns."},
        {"id": 7, "description": "As a female user, I want to log specific symptoms for detailed tracking."},
        {"id": 8, "description": "As a female administrator, I want statistics on wellness patterns across the platform."},
        {"id": 9, "description": "As a female user, I want notifications about upcoming cycles or changes in patterns."},
        {"id": 10, "description": "As a female administrator, I want interface customization options."},
        {"id": 11, "description": "As a female user, I want access to personal care recommendations based on my cycle."},
        {"id": 12, "description": "As a female administrator, I want to view user satisfaction statistics."},
        {"id": 13, "description": "As a female user, I want access to a full history of my past cycles."},
        {"id": 14, "description": "As a female administrator, I want to manage the app’s integration with other health devices."},
        {"id": 15, "description": "As a female user, I want to mark certain days or symptoms as important for later review."},
        {"id": 16, "description": "As a female administrator, I want tools to analyze trends in user wellness."},
        {"id": 17, "description": "As a female user, I want customizable alert options based on my preferences."},
        {"id": 18, "description": "As a female administrator, I want reports on usage and effectiveness of the app."},
        {"id": 19, "description": "As a female user, I want to receive support messages and product recommendations based on my cycle."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Finance Management App for SMEs
Guidelines: ['Personal data privacy and security', 'User-friendly interface', 'Personalized reminders', 'Well-being reports', 'Cross-platform access']
Dimensiones asociadas (7): ['Operational Benefits', 'Technical Context', 'Risks', 'System-related Benefits', 'Stakeholder Benefits', 'Costs', 'Benefits']

Subdimensiones esenciales (ID: Nombre):
  210: Object usage for a particular scenario
  211: Actor usage for a particular scenario
  52: Contribution to user task
  228: Number of use cases inherited by this use case
  224: Number of use cases this use case includes
  225: Number of use cases that includes this use case
  226: Number of use cases this use case extends
  227: Number of use cases that extend this use case
  197: Experience on the area
  2: Linkage to overall system goals
  37: Relevance to stakeholders' goals
  236: Dependencies due to downstream activities
  105: Impact on essential non-functional requirements
  198: Experience

In [52]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-044
project = {
    "name": "Vehicle Rental Application",
    "Description": "An app that simplifies vehicle rental, allowing users to choose options based on preferences and manage the reservation process.",
    "guidelines": [
        "Secure client info access",
        "Response time reports",
        "Communication channel integration",
        "Response personalization",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As a customer, I want to submit inquiries and requests online to receive assistance."},
        {"id": 2, "description": "As an agent, I want to manage customer cases and follow up on them."},
        {"id": 3, "description": "As an administrator, I want reports on the response time for each case."},
        {"id": 4, "description": "As an agent, I want options to escalate cases to higher levels."},
        {"id": 5, "description": "As a customer, I want to receive update notifications on my case."},
        {"id": 6, "description": "As an administrator, I want to see customer satisfaction statistics."},
        {"id": 7, "description": "As an agent, I want to view each customer's inquiry history to provide personalized responses."},
        {"id": 8, "description": "As a customer, I want access to a knowledge base to solve common issues."},
        {"id": 9, "description": "As an administrator, I want to manage agent access permissions to customer data."},
        {"id": 10, "description": "As an agent, I want to conduct satisfaction surveys after resolving a case."},
        {"id": 11, "description": "As a customer, I want the option to rate the support I received."},
        {"id": 12, "description": "As an administrator, I want reports on the most frequent cases and their solutions."},
        {"id": 13, "description": "As an agent, I want to receive alerts about newly assigned cases."},
        {"id": 14, "description": "As an administrator, I want platform customization options based on the company."},
        {"id": 15, "description": "As a customer, I want the option to chat in real time with an agent."},
        {"id": 16, "description": "As an administrator, I want an analysis of the customer service team’s effectiveness."},
        {"id": 17, "description": "As an agent, I want access to response templates to speed up service."},
        {"id": 18, "description": "As a customer, I want fast and detailed responses to my inquiries."},
        {"id": 19, "description": "As an administrator, I want to generate global customer satisfaction reports."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Vehicle Rental Application
Guidelines: ['Secure client info access', 'Response time reports', 'Communication channel integration', 'Response personalization', 'Scalability']
Dimensiones asociadas (7): ['Business Context', 'Operational Benefits', 'Costs', 'System-related Benefits', 'Risks', 'Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  172: Time to market
  183: Integration to external systems
  2: Linkage to overall system goals
  52: Contribution to user task
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  110: Technical risk in current system
  111: Technical risk in proposed system
  16: Customer loyalty
  18: Additional customer sales
  19: Extra cost customer will spend
  63: Customer relations

Requisitos con sus subdimensiones y PriorityScore:

1. As a customer, I want to submit inqui

In [53]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-045
project = {
    "name": "Shopping and Discounts Application",
    "Description": "An app that provides tools to improve users’ mental well-being, including meditation sessions, mood tracking, and breathing exercises.",
    "guidelines": [
        "Personal data security",
        "Contract management",
        "Performance reports",
        "Work event notifications",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As an administrator, I want to securely manage each employee's personal information."},
        {"id": 2, "description": "As a manager, I want to receive alerts for contract expirations."},
        {"id": 3, "description": "As an administrator, I want options to update employee benefits."},
        {"id": 4, "description": "As a manager, I want to view performance reports by department."},
        {"id": 5, "description": "As an administrator, I want to manage access permissions based on roles."},
        {"id": 6, "description": "As an employee, I want to view a history of my performance evaluations."},
        {"id": 7, "description": "As a manager, I want options to plan promotions and salary increases."},
        {"id": 8, "description": "As an administrator, I want to automatically manage contract renewals."},
        {"id": 9, "description": "As an employee, I want to receive notifications about company events."},
        {"id": 10, "description": "As an administrator, I want reports on employee turnover by department."},
        {"id": 11, "description": "As a manager, I want access to a database of internal candidates for promotions."},
        {"id": 12, "description": "As an administrator, I want to manage the company's organizational structure."},
        {"id": 13, "description": "As an employee, I want to receive reminders for my performance reviews."},
        {"id": 14, "description": "As an administrator, I want statistics on average time spent in each position."},
        {"id": 15, "description": "As a manager, I want to conduct annual performance evaluations."},
        {"id": 16, "description": "As an administrator, I want to export employee information reports."},
        {"id": 17, "description": "As an employee, I want access to a summary of my employment benefits."},
        {"id": 18, "description": "As an administrator, I want to manage platform access based on work location."},
        {"id": 19, "description": "As a manager, I want an analysis of employee job satisfaction."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Shopping and Discounts Application
Guidelines: ['Personal data security', 'Contract management', 'Performance reports', 'Work event notifications', 'Scalability']
Dimensiones asociadas (7): ['System-related Benefits', 'Risks', 'Operational Benefits', 'Benefits', 'Market-related Benefits', 'Project-related Benefits', 'Stakeholder Benefits']

Subdimensiones esenciales (ID: Nombre):
  2: Linkage to overall system goals
  30: Relevance to project success
  31: Importance to release goal
  33: Feature contribution to project vision
  37: Relevance to stakeholders' goals
  52: Contribution to user task
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  110: Technical risk in current system
  111: Technical risk in proposed system
  36: Value creation for developer
  53: Support for main usage scenario
  56: Cost reduction of IT operation
  109: Loss of confidential data
  3: Ease of use

Requisitos con sus subdimensiones y PriorityS

In [54]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-046
project = {
    "name": "Online Medical Appointment Booking System",
    "Description": "A system that enables patients to book medical appointments online, receive reminders, and access their visit history.",
    "guidelines": [
        "Real-time data access",
        "Defect reports",
        "Production system integration",
        "Information security",
        "Scalability"
    ],
    "requirements": [
        {"id": 1, "description": "As a supervisor, I want to monitor product quality in real time."},
        {"id": 2, "description": "As an operator, I want to record defects observed during the production process."},
        {"id": 3, "description": "As an administrator, I want to generate reports of defective products by shift."},
        {"id": 4, "description": "As a supervisor, I want to receive alerts for critical defects detected."},
        {"id": 5, "description": "As an administrator, I want to manage access permissions for quality data."},
        {"id": 6, "description": "As an operator, I want to receive notifications of updated quality procedures."},
        {"id": 7, "description": "As a supervisor, I want to view defect charts by production batch."},
        {"id": 8, "description": "As an administrator, I want options to integrate with other quality control systems."},
        {"id": 9, "description": "As an operator, I want access to specific quality instructions for each product."},
        {"id": 10, "description": "As a supervisor, I want to compare quality across different production shifts."},
        {"id": 11, "description": "As an administrator, I want to view statistics on compliance with quality standards."},
        {"id": 12, "description": "As an operator, I want to record observations about quality procedures."},
        {"id": 13, "description": "As a supervisor, I want reports on the effectiveness of corrective actions."},
        {"id": 14, "description": "As an administrator, I want to manage product traceability in the system."},
        {"id": 15, "description": "As a supervisor, I want to access a quality history for each production line."},
        {"id": 16, "description": "As an administrator, I want to generate customer satisfaction reports related to quality."},
        {"id": 17, "description": "As an operator, I want to receive alerts for pending quality reviews."},
        {"id": 18, "description": "As a supervisor, I want access to internal and external audit reports."},
        {"id": 19, "description": "As an administrator, I want options to export quality data for external analysis."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Online Medical Appointment Booking System
Guidelines: ['Real-time data access', 'Defect reports', 'Production system integration', 'Information security', 'Scalability']
Dimensiones asociadas (7): ['Risks', 'Business Context', 'System-related Benefits', 'Operational Benefits', 'Costs', 'Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  183: Integration to external systems
  2: Linkage to overall system goals
  172: Time to market
  52: Contribution to user task
  56: Cost reduction of IT operation
  110: Technical risk in current system
  111: Technical risk in proposed system
  186: IT departments technical guidelines
  26: Feature influence on buying decision
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  175: Importance for product architecture
  101: Quality implementation effort

Requisitos con sus subdimensiones y PriorityScore:

1

In [55]:
#Imports and helper functions
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math

def clean_text(text):
    # Convert to lowercase, remove punctuation, and return a list of words
    text = text.lower()
    # preserve accents and ñ
    text = re.sub(r'[^a-z0-9áéíóúüñ]+', ' ', text)
    return text.split()

def get_keywords(subdim):
    # Return the list of keywords in lowercase for a sub-dimension
    return [kw.lower() for kw in subdim["keywords"]]

    #Dimensions and  Subdimensions

dimensions = [
    # Benefits
    {"dimension": "Benefits", "subdim_id": 1, "name": "Business value", "keywords": ["business value", "business importance", "gain for organization"]},

    # System-related Benefits
    {"dimension": "System-related Benefits", "subdim_id": 2, "name": "Linkage to overall system goals", "keywords": ["system goals", "alignment"]},
    {"dimension": "System-related Benefits", "subdim_id": 3, "name": "Ease of use", "keywords": ["ease of use", "convenience", "user friendly"]},
    {"dimension": "System-related Benefits", "subdim_id": 4, "name": "Scalability", "keywords": ["scalable", "scale", "load"]},
    {"dimension": "System-related Benefits", "subdim_id": 5, "name": "Sustainability of solution", "keywords": ["sustainable", "long-term"]},
    {"dimension": "System-related Benefits", "subdim_id": 6, "name": "Changeable solution", "keywords": ["changeable", "adaptable"]},
    {"dimension": "System-related Benefits", "subdim_id": 7, "name": "Uniform solution", "keywords": ["uniform", "consistent"]},
    {"dimension": "System-related Benefits", "subdim_id": 8, "name": "Performance", "keywords": ["performance", "speed", "efficiency"]},
    {"dimension": "System-related Benefits", "subdim_id": 9, "name": "Stability", "keywords": ["stable", "reliable"]},
    {"dimension": "System-related Benefits", "subdim_id": 10, "name": "Security", "keywords": ["secure", "security"]},
    {"dimension": "System-related Benefits", "subdim_id": 11, "name": "Integrity", "keywords": ["integrity", "data quality"]},
    {"dimension": "System-related Benefits", "subdim_id": 12, "name": "Availability", "keywords": ["available", "uptime"]},
    {"dimension": "System-related Benefits", "subdim_id": 13, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "System-related Benefits", "subdim_id": 14, "name": "Accuracy", "keywords": ["accurate", "precision"]},

    # Market-related Benefits
    {"dimension": "Market-related Benefits", "subdim_id": 15, "name": "Product Market value", "keywords": ["market value", "market share", "revenue"]},
    {"dimension": "Market-related Benefits", "subdim_id": 16, "name": "Customer loyalty", "keywords": ["loyalty", "retention"]},
    {"dimension": "Market-related Benefits", "subdim_id": 17, "name": "Marketability", "keywords": ["marketability", "ability to sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 18, "name": "Additional customer sales", "keywords": ["upsell", "cross sell"]},
    {"dimension": "Market-related Benefits", "subdim_id": 19, "name": "Extra cost customer will spend", "keywords": ["premium", "additional spend"]},
    {"dimension": "Market-related Benefits", "subdim_id": 20, "name": "Market percentage", "keywords": ["market share", "percentage"]},
    {"dimension": "Market-related Benefits", "subdim_id": 21, "name": "Creation of competitive advantage", "keywords": ["competitive advantage", "differentiation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 22, "name": "Status of competitors", "keywords": ["competitors", "competitive landscape"]},
    {"dimension": "Market-related Benefits", "subdim_id": 23, "name": "Innovativeness", "keywords": ["innovative", "innovation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 24, "name": "Market technology trends", "keywords": ["trends", "technology trends"]},
    {"dimension": "Market-related Benefits", "subdim_id": 25, "name": "Brand protection", "keywords": ["brand", "reputation"]},
    {"dimension": "Market-related Benefits", "subdim_id": 26, "name": "Feature influence on buying decision", "keywords": ["buying decision", "purchase decision"]},
    {"dimension": "Market-related Benefits", "subdim_id": 27, "name": "Resalable solution", "keywords": ["resell", "white label"]},
    {"dimension": "Market-related Benefits", "subdim_id": 28, "name": "Long term Product strategy", "keywords": ["product strategy", "roadmap"]},
    {"dimension": "Market-related Benefits", "subdim_id": 29, "name": "Fit with other products", "keywords": ["product fit", "ecosystem"]},

    # Project-related Benefits
    {"dimension": "Project-related Benefits", "subdim_id": 30, "name": "Relevance to project success", "keywords": ["project success", "critical success"]},
    {"dimension": "Project-related Benefits", "subdim_id": 31, "name": "Importance to release goal", "keywords": ["release goal", "sprint goal"]},
    {"dimension": "Project-related Benefits", "subdim_id": 32, "name": "Release theme", "keywords": ["theme", "release theme"]},
    {"dimension": "Project-related Benefits", "subdim_id": 33, "name": "Feature contribution to project vision", "keywords": ["project vision", "mission"]},
    {"dimension": "Project-related Benefits", "subdim_id": 34, "name": "Synergy effects", "keywords": ["synergy", "combined effect"]},

    # (Particular) Stakeholder Benefits
    {"dimension": "Stakeholder Benefits", "subdim_id": 35, "name": "End user value", "keywords": ["user value", "user satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 36, "name": "Value creation for developer", "keywords": ["developer value", "engineer satisfaction"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 37, "name": "Relevance to stakeholders' goals", "keywords": ["stakeholder goals", "alignment"]},
    {"dimension": "Stakeholder Benefits", "subdim_id": 38, "name": "Fit with skills", "keywords": ["skills", "training"]},

    # Financial Benefits
    {"dimension": "Financial Benefits", "subdim_id": 39, "name": "Return on investment", "keywords": ["roi", "return on investment"]},
    {"dimension": "Financial Benefits", "subdim_id": 40, "name": "Net present value", "keywords": ["npv", "net present value"]},
    {"dimension": "Financial Benefits", "subdim_id": 41, "name": "Internal rate of return", "keywords": ["irr", "internal rate of return"]},
    {"dimension": "Financial Benefits", "subdim_id": 42, "name": "Payback period", "keywords": ["payback", "payback period"]},

    # (Business) Strategy-related Benefits
    {"dimension": "Strategy-related Benefits", "subdim_id": 43, "name": "Importance to business goals", "keywords": ["business goals", "strategic goals"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 44, "name": "Criticality to mission success", "keywords": ["mission critical", "critical"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 45, "name": "Long term strategic value", "keywords": ["strategic value", "long term"]},
    {"dimension": "Strategy-related Benefits", "subdim_id": 46, "name": "Tactical usefulness", "keywords": ["tactical", "short term"]},

    # Customer Benefits
    {"dimension": "Customer Benefits", "subdim_id": 47, "name": "Efficiency gains for customer", "keywords": ["customer efficiency", "time saving"]},
    {"dimension": "Customer Benefits", "subdim_id": 48, "name": "Competitive gains for customer", "keywords": ["customer advantage", "competitive edge"]},
    {"dimension": "Customer Benefits", "subdim_id": 49, "name": "Intermediary satisfaction", "keywords": ["intermediary", "partner satisfaction"]},

    # Operational (Business) Performance Benefits
    {"dimension": "Operational Benefits", "subdim_id": 50, "name": "Organizational effectiveness", "keywords": ["effectiveness", "productivity"]},
    {"dimension": "Operational Benefits", "subdim_id": 51, "name": "Fit with business processes", "keywords": ["business processes", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 52, "name": "Contribution to user task", "keywords": ["user task", "job"]},
    {"dimension": "Operational Benefits", "subdim_id": 53, "name": "Support for main usage scenario", "keywords": ["main scenario", "primary use"]},
    {"dimension": "Operational Benefits", "subdim_id": 54, "name": "Cost saving", "keywords": ["cost saving", "reduction"]},
    {"dimension": "Operational Benefits", "subdim_id": 55, "name": "Economies of production", "keywords": ["economies", "scale"]},
    {"dimension": "Operational Benefits", "subdim_id": 56, "name": "Cost reduction of IT operation", "keywords": ["it cost", "operation cost"]},
    {"dimension": "Operational Benefits", "subdim_id": 57, "name": "Process efficiency", "keywords": ["process efficiency", "workflow"]},
    {"dimension": "Operational Benefits", "subdim_id": 58, "name": "Speed", "keywords": ["speed", "fast"]},
    {"dimension": "Operational Benefits", "subdim_id": 59, "name": "Throughput", "keywords": ["throughput", "capacity"]},
    {"dimension": "Operational Benefits", "subdim_id": 60, "name": "Countermeasure benefit", "keywords": ["countermeasure", "mitigation"]},
    {"dimension": "Operational Benefits", "subdim_id": 61, "name": "Inbound logistics", "keywords": ["logistics", "supply chain"]},
    {"dimension": "Operational Benefits", "subdim_id": 62, "name": "Supplier relations", "keywords": ["supplier", "vendor"]},
    {"dimension": "Operational Benefits", "subdim_id": 63, "name": "Customer relations", "keywords": ["customer relations", "client"]},
    {"dimension": "Operational Benefits", "subdim_id": 64, "name": "Competitor relations", "keywords": ["competitor", "rival"]},
    {"dimension": "Operational Benefits", "subdim_id": 65, "name": "Business innovation", "keywords": ["innovation", "disruption"]},
    {"dimension": "Operational Benefits", "subdim_id": 66, "name": "Deliveries", "keywords": ["delivery", "shipment"]},
    {"dimension": "Operational Benefits", "subdim_id": 67, "name": "Third party relations", "keywords": ["third party", "partner"]},
    {"dimension": "Operational Benefits", "subdim_id": 68, "name": "Marketing support", "keywords": ["marketing", "promotion"]},
    {"dimension": "Operational Benefits", "subdim_id": 69, "name": "Decision making", "keywords": ["decision", "choice"]},
    {"dimension": "Operational Benefits", "subdim_id": 70, "name": "Learning and knowledge", "keywords": ["learning", "knowledge"]},
    {"dimension": "Operational Benefits", "subdim_id": 71, "name": "Organization culture", "keywords": ["culture", "values"]},
    {"dimension": "Operational Benefits", "subdim_id": 72, "name": "Information", "keywords": ["information", "data"]},
    {"dimension": "Operational Benefits", "subdim_id": 73, "name": "Technology", "keywords": ["technology", "tools"]},
    {"dimension": "Operational Benefits", "subdim_id": 74, "name": "Strategy formulation", "keywords": ["strategy", "planning"]},
    {"dimension": "Operational Benefits", "subdim_id": 75, "name": "Communication", "keywords": ["communication", "collaboration"]},
    {"dimension": "Operational Benefits", "subdim_id": 76, "name": "Flow of products", "keywords": ["flow", "supply"]},
    {"dimension": "Operational Benefits", "subdim_id": 77, "name": "Control and follow-up", "keywords": ["control", "reporting"]},
    {"dimension": "Operational Benefits", "subdim_id": 78, "name": "Change management", "keywords": ["change", "adaptation"]},
    {"dimension": "Operational Benefits", "subdim_id": 79, "name": "Integration and coordination", "keywords": ["integration", "coordination"]},
    {"dimension": "Operational Benefits", "subdim_id": 80, "name": "Flexibility", "keywords": ["flexible", "adaptable"]},

    # Costs - Generic Cost Types
    {"dimension": "Costs", "subdim_id": 81, "name": "Life-cycle costs", "keywords": ["lifecycle", "total cost"]},
    {"dimension": "Costs", "subdim_id": 82, "name": "Total ownership cost", "keywords": ["tco", "total cost"]},
    {"dimension": "Costs", "subdim_id": 83, "name": "Money", "keywords": ["money", "finances", "budget"]},
    {"dimension": "Costs", "subdim_id": 84, "name": "Time", "keywords": ["time", "duration"]},
    {"dimension": "Costs", "subdim_id": 85, "name": "Labor", "keywords": ["labor", "workforce"]},
    {"dimension": "Costs", "subdim_id": 86, "name": "Overhead", "keywords": ["overhead", "indirect"]},
    {"dimension": "Costs", "subdim_id": 87, "name": "Hardware unit costs", "keywords": ["hardware", "equipment"]},
    {"dimension": "Costs", "subdim_id": 88, "name": "Travel", "keywords": ["travel", "transport"]},
    {"dimension": "Costs", "subdim_id": 89, "name": "Material", "keywords": ["material", "supplies"]},

    # Post-development Time related Costs
    {"dimension": "Costs", "subdim_id": 90, "name": "Maintenance cost", "keywords": ["maintenance", "support"]},
    {"dimension": "Costs", "subdim_id": 91, "name": "Support costs", "keywords": ["support", "helpdesk"]},
    {"dimension": "Costs", "subdim_id": 92, "name": "Operational costs", "keywords": ["operations", "running"]},
    {"dimension": "Costs", "subdim_id": 93, "name": "Shipping", "keywords": ["shipping", "delivery"]},

    # Development Time related Costs
    {"dimension": "Costs", "subdim_id": 94, "name": "Task size", "keywords": ["size", "scope"]},
    {"dimension": "Costs", "subdim_id": 95, "name": "Documentation cost", "keywords": ["documentation", "writing"]},
    {"dimension": "Costs", "subdim_id": 96, "name": "Functional implementation effort", "keywords": ["functional", "features"]},
    {"dimension": "Costs", "subdim_id": 97, "name": "Technical implementation effort", "keywords": ["technical", "coding"]},
    {"dimension": "Costs", "subdim_id": 98, "name": "Integration testing cost", "keywords": ["integration", "testing"]},
    {"dimension": "Costs", "subdim_id": 99, "name": "User acceptance testing cost", "keywords": ["uat", "acceptance"]},
    {"dimension": "Costs", "subdim_id": 100, "name": "Cost for fixing defects", "keywords": ["defects", "bugs"]},
    {"dimension": "Costs", "subdim_id": 101, "name": "Quality implementation effort", "keywords": ["quality", "attributes"]},
    {"dimension": "Costs", "subdim_id": 102, "name": "Risk mitigation effort", "keywords": ["mitigation", "reduction"]},
    {"dimension": "Costs", "subdim_id": 103, "name": "Product costs", "keywords": ["product", "development"]},

    # Risks - Technical & Implementation-related Risks
    {"dimension": "Risks", "subdim_id": 104, "name": "Severe redesign of architecture", "keywords": ["redesign", "architecture"]},
    {"dimension": "Risks", "subdim_id": 105, "name": "Impact on essential non-functional requirements", "keywords": ["non-functional", "requirements"]},
    {"dimension": "Risks", "subdim_id": 106, "name": "Database risks", "keywords": ["database", "storage"]},
    {"dimension": "Risks", "subdim_id": 107, "name": "Performance risks", "keywords": ["performance", "speed"]},
    {"dimension": "Risks", "subdim_id": 108, "name": "Risk of maintenance", "keywords": ["maintenance", "upkeep"]},
    {"dimension": "Risks", "subdim_id": 109, "name": "Loss of confidential data", "keywords": ["confidential", "data loss"]},
    {"dimension": "Risks", "subdim_id": 110, "name": "Technical risk in current system", "keywords": ["current", "legacy"]},
    {"dimension": "Risks", "subdim_id": 111, "name": "Technical risk in proposed system", "keywords": ["proposed", "new"]},
    {"dimension": "Risks", "subdim_id": 112, "name": "Technical Complexity", "keywords": ["complex", "difficult"]},
    {"dimension": "Risks", "subdim_id": 113, "name": "Implementation technology risks", "keywords": ["technology", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 114, "name": "Scope loss", "keywords": ["scope", "creep"]},
    {"dimension": "Risks", "subdim_id": 115, "name": "Risk of buggy implementation", "keywords": ["buggy", "defects"]},
    {"dimension": "Risks", "subdim_id": 116, "name": "Development risk", "keywords": ["development", "coding"]},
    {"dimension": "Risks", "subdim_id": 117, "name": "Difficulty of programming language", "keywords": ["language", "syntax"]},
    {"dimension": "Risks", "subdim_id": 118, "name": "Large Size", "keywords": ["large", "big"]},
    {"dimension": "Risks", "subdim_id": 119, "name": "Tedium", "keywords": ["tedious", "boring"]},
    {"dimension": "Risks", "subdim_id": 120, "name": "Uncertainty", "keywords": ["uncertain", "unknown"]},
    {"dimension": "Risks", "subdim_id": 121, "name": "Novelty", "keywords": ["novel", "new"]},
    {"dimension": "Risks", "subdim_id": 122, "name": "Number of people involved", "keywords": ["people", "team"]},
    {"dimension": "Risks", "subdim_id": 123, "name": "Organizational constraints", "keywords": ["organizational", "constraints"]},
    {"dimension": "Risks", "subdim_id": 124, "name": "Ease of realization (technical feasibility)", "keywords": ["feasibility", "technical"]},
    {"dimension": "Risks", "subdim_id": 125, "name": "External risks", "keywords": ["external", "outside"]},

    # Requirements Status related Risks
    {"dimension": "Risks", "subdim_id": 126, "name": "Vague story", "keywords": ["vague", "unclear"]},
    {"dimension": "Risks", "subdim_id": 127, "name": "Changes from business perspective", "keywords": ["business", "changes"]},
    {"dimension": "Risks", "subdim_id": 128, "name": "Changes from technical perspective", "keywords": ["technical", "changes"]},
    {"dimension": "Risks", "subdim_id": 129, "name": "Market changes", "keywords": ["market", "changes"]},
    {"dimension": "Risks", "subdim_id": 130, "name": "Legislative changes", "keywords": ["legislative", "law"]},
    {"dimension": "Risks", "subdim_id": 131, "name": "Users change", "keywords": ["users", "change"]},
    {"dimension": "Risks", "subdim_id": 132, "name": "Requirements become more clear", "keywords": ["clear", "clarify"]},

    # Business-related Risks
    {"dimension": "Risks", "subdim_id": 133, "name": "Productivity loss", "keywords": ["productivity", "loss"]},
    {"dimension": "Risks", "subdim_id": 134, "name": "Loss of reputation", "keywords": ["reputation", "image"]},
    {"dimension": "Risks", "subdim_id": 135, "name": "Loss of customers", "keywords": ["customers", "loss"]},
    {"dimension": "Risks", "subdim_id": 136, "name": "Security related risks", "keywords": ["security", "misuse"]},
    {"dimension": "Risks", "subdim_id": 137, "name": "Safety risks", "keywords": ["safety", "hazard"]},
    {"dimension": "Risks", "subdim_id": 138, "name": "Ease of realization (economic feasibility)", "keywords": ["economic", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 139, "name": "Ease of realization (social feasibility)", "keywords": ["social", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 140, "name": "Ease of realization (political feasibility)", "keywords": ["political", "feasibility"]},
    {"dimension": "Risks", "subdim_id": 141, "name": "Market uncertainty", "keywords": ["market", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 142, "name": "Sales barriers", "keywords": ["sales", "barriers"]},
    {"dimension": "Risks", "subdim_id": 143, "name": "Commercial concerns", "keywords": ["commercial", "concerns"]},

    # Project-related Risks
    {"dimension": "Risks", "subdim_id": 144, "name": "Overrun", "keywords": ["overrun", "schedule"]},
    {"dimension": "Risks", "subdim_id": 145, "name": "Fixing cost", "keywords": ["fixing", "cost"]},
    {"dimension": "Risks", "subdim_id": 146, "name": "Implementation Process risk", "keywords": ["process", "implementation"]},
    {"dimension": "Risks", "subdim_id": 147, "name": "Environmental factor", "keywords": ["environmental", "context"]},
    {"dimension": "Risks", "subdim_id": 148, "name": "Project duration", "keywords": ["duration", "length"]},
    {"dimension": "Risks", "subdim_id": 149, "name": "Project dependencies", "keywords": ["dependencies", "interdependencies"]},
    {"dimension": "Risks", "subdim_id": 150, "name": "Impediment of attaining the requirement", "keywords": ["impediment", "blocker"]},

    # Stakeholder-related Risks
    {"dimension": "Risks", "subdim_id": 151, "name": "Personnel risks", "keywords": ["personnel", "staff"]},
    {"dimension": "Risks", "subdim_id": 152, "name": "Estimation risks", "keywords": ["estimation", "guess"]},
    {"dimension": "Risks", "subdim_id": 153, "name": "Risks of acceptance", "keywords": ["acceptance", "adoption"]},
    {"dimension": "Risks", "subdim_id": 154, "name": "Decision uncertainty", "keywords": ["decision", "uncertainty"]},
    {"dimension": "Risks", "subdim_id": 155, "name": "Part time team member use", "keywords": ["part time", "fractional"]},

    # Penalties & Penalty Avoidance
    {"dimension": "Penalties", "subdim_id": 156, "name": "Negative value", "keywords": ["negative", "loss"]},
    {"dimension": "Penalties", "subdim_id": 157, "name": "Harm avoidance", "keywords": ["harm", "avoid"]},

    # Operational (Business) Performance related Penalties
    {"dimension": "Penalties", "subdim_id": 158, "name": "Complicated workaround", "keywords": ["workaround", "complicated"]},
    {"dimension": "Penalties", "subdim_id": 159, "name": "Penalty in operation", "keywords": ["operation", "penalty"]},

    # Product- & Market-related Penalties
    {"dimension": "Penalties", "subdim_id": 160, "name": "Detraction from product's value", "keywords": ["detraction", "value"]},
    {"dimension": "Penalties", "subdim_id": 161, "name": "Market share aspect", "keywords": ["market share", "penalty"]},

    # Customer Penalties
    {"dimension": "Penalties", "subdim_id": 162, "name": "Negative value for customer", "keywords": ["customer", "negative"]},
    {"dimension": "Penalties", "subdim_id": 163, "name": "Customer dissatisfaction", "keywords": ["dissatisfaction", "unhappy"]},

    # Financial Penalties
    {"dimension": "Penalties", "subdim_id": 164, "name": "Cost of not implementing", "keywords": ["not implementing", "cost"]},
    {"dimension": "Penalties", "subdim_id": 165, "name": "Financial penalty", "keywords": ["financial", "penalty"]},
    {"dimension": "Penalties", "subdim_id": 166, "name": "Legal mandate", "keywords": ["legal", "compliance"]},
    {"dimension": "Penalties", "subdim_id": 167, "name": "Promised commitment", "keywords": ["promised", "commitment"]},

    # Stakeholder Penalties
    {"dimension": "Penalties", "subdim_id": 168, "name": "Stakeholder Dissatisfaction", "keywords": ["stakeholder", "dissatisfaction"]},

    # Business Context - External Context
    {"dimension": "Business Context", "subdim_id": 169, "name": "After sale support", "keywords": ["support", "after sale"]},
    {"dimension": "Business Context", "subdim_id": 170, "name": "Customer demand", "keywords": ["demand", "customer"]},
    {"dimension": "Business Context", "subdim_id": 171, "name": "Formal governmental demand", "keywords": ["governmental", "formal"]},

    # Time & Schedule
    {"dimension": "Business Context", "subdim_id": 172, "name": "Time to market", "keywords": ["time", "market"]},

    # Utilization / Usage
    {"dimension": "Business Context", "subdim_id": 173, "name": "Keep legacy system alive", "keywords": ["legacy", "system"]},
    {"dimension": "Business Context", "subdim_id": 174, "name": "Impact on Maintenance", "keywords": ["maintenance", "impact"]},
    {"dimension": "Business Context", "subdim_id": 175, "name": "Importance for product architecture", "keywords": ["architecture", "product"]},
    {"dimension": "Business Context", "subdim_id": 176, "name": "Technical debt", "keywords": ["technical debt", "debt"]},
    {"dimension": "Business Context", "subdim_id": 177, "name": "Impact on long-term Evolution", "keywords": ["evolution", "long term"]},
    {"dimension": "Business Context", "subdim_id": 178, "name": "Short term Architectural impact", "keywords": ["architectural", "short term"]},
    {"dimension": "Business Context", "subdim_id": 179, "name": "Infrastructure", "keywords": ["infrastructure", "criticality"]},
    {"dimension": "Business Context", "subdim_id": 180, "name": "Preferred operating architecture", "keywords": ["operating", "architecture"]},
    {"dimension": "Business Context", "subdim_id": 181, "name": "Technology opportunities", "keywords": ["technology", "opportunities"]},
    {"dimension": "Business Context", "subdim_id": 182, "name": "Technology should support current functionality", "keywords": ["current", "functionality"]},
    {"dimension": "Business Context", "subdim_id": 183, "name": "Integration to external systems", "keywords": ["integration", "external"]},
    {"dimension": "Business Context", "subdim_id": 184, "name": "System value of a feature", "keywords": ["system", "value"]},
    {"dimension": "Business Context", "subdim_id": 185, "name": "Use case weight", "keywords": ["use case", "weight"]},
    {"dimension": "Business Context", "subdim_id": 186, "name": "IT departments technical guidelines", "keywords": ["it", "guidelines"]},
    {"dimension": "Business Context", "subdim_id": 187, "name": "Technical priority", "keywords": ["technical", "priority"]},
    {"dimension": "Business Context", "subdim_id": 188, "name": "Frequency of use", "keywords": ["frequency", "use"]},
    {"dimension": "Business Context", "subdim_id": 189, "name": "Actor priority", "keywords": ["actor", "priority"]},
    {"dimension": "Business Context", "subdim_id": 190, "name": "Development Lead time", "keywords": ["lead time", "development"]},
    {"dimension": "Business Context", "subdim_id": 191, "name": "Delivery date", "keywords": ["delivery", "date"]},
    {"dimension": "Business Context", "subdim_id": 192, "name": "Project deadline", "keywords": ["deadline", "project"]},

    # Stakeholders
    {"dimension": "Business Context", "subdim_id": 193, "name": "Originator of requirement", "keywords": ["originator", "requirement"]},
    {"dimension": "Business Context", "subdim_id": 194, "name": "Stakeholder agreement", "keywords": ["agreement", "stakeholder"]},
    {"dimension": "Business Context", "subdim_id": 195, "name": "Likelihood of success", "keywords": ["likelihood", "success"]},

    # Technical Context & Requirements Characteristics - Engineering Resources
    {"dimension": "Technical Context", "subdim_id": 196, "name": "Familiarity of the life-cycle model", "keywords": ["lifecycle", "model"]},
    {"dimension": "Technical Context", "subdim_id": 197, "name": "Experience on the area", "keywords": ["experience", "area"]},
    {"dimension": "Technical Context", "subdim_id": 198, "name": "Experience on development methodology", "keywords": ["methodology", "development"]},
    {"dimension": "Technical Context", "subdim_id": 199, "name": "Ability of analyzer", "keywords": ["ability", "analyzer"]},
    {"dimension": "Technical Context", "subdim_id": 200, "name": "Motivation of the team", "keywords": ["motivation", "team"]},
    {"dimension": "Technical Context", "subdim_id": 201, "name": "Balanced workload", "keywords": ["workload", "balanced"]},
    {"dimension": "Technical Context", "subdim_id": 202, "name": "Velocity", "keywords": ["velocity", "speed"]},
    {"dimension": "Technical Context", "subdim_id": 203, "name": "Resources for specification", "keywords": ["resources", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 204, "name": "Functional resources", "keywords": ["functional", "resources"]},
    {"dimension": "Technical Context", "subdim_id": 205, "name": "Analysis and design", "keywords": ["analysis", "design"]},
    {"dimension": "Technical Context", "subdim_id": 206, "name": "Implementation", "keywords": ["implementation", "development"]},
    {"dimension": "Technical Context", "subdim_id": 207, "name": "Testing", "keywords": ["testing", "qa"]},
    {"dimension": "Technical Context", "subdim_id": 208, "name": "User interface", "keywords": ["ui", "ux"]},
    {"dimension": "Technical Context", "subdim_id": 209, "name": "Research", "keywords": ["research", "r&d"]},

    # Requirements Status & Qualities
    {"dimension": "Technical Context", "subdim_id": 210, "name": "Object usage for a particular scenario", "keywords": ["object", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 211, "name": "Actor usage for a particular scenario", "keywords": ["actor", "scenario"]},
    {"dimension": "Technical Context", "subdim_id": 212, "name": "Reuse potential", "keywords": ["reuse", "potential"]},
    {"dimension": "Technical Context", "subdim_id": 213, "name": "Number of times use case appears", "keywords": ["use case", "appears"]},

    # Dependencies
    {"dimension": "Technical Context", "subdim_id": 214, "name": "Readiness for implementation", "keywords": ["readiness", "implementation"]},
    {"dimension": "Technical Context", "subdim_id": 215, "name": "Sufficient detail in specification", "keywords": ["detail", "specification"]},
    {"dimension": "Technical Context", "subdim_id": 216, "name": "Modifiability", "keywords": ["modifiable", "change"]},
    {"dimension": "Technical Context", "subdim_id": 217, "name": "Traceability", "keywords": ["traceable", "track"]},
    {"dimension": "Technical Context", "subdim_id": 218, "name": "Testability", "keywords": ["testable", "testing"]},
    {"dimension": "Technical Context", "subdim_id": 219, "name": "Completeness", "keywords": ["complete", "whole"]},
    {"dimension": "Technical Context", "subdim_id": 220, "name": "Consistency", "keywords": ["consistent", "uniform"]},
    {"dimension": "Technical Context", "subdim_id": 221, "name": "Understandability", "keywords": ["understandable", "clear"]},
    {"dimension": "Technical Context", "subdim_id": 222, "name": "Within Scope", "keywords": ["scope", "within"]},
    {"dimension": "Technical Context", "subdim_id": 223, "name": "Non-redundant", "keywords": ["redundant", "duplicate"]},

    # Technology, Infrastructure & Architecture
    {"dimension": "Technical Context", "subdim_id": 224, "name": "Number of use cases this use case includes", "keywords": ["includes", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 225, "name": "Number of use cases that includes this use case", "keywords": ["included by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 226, "name": "Number of use cases this use case extends", "keywords": ["extends", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 227, "name": "Number of use cases that extend this use case", "keywords": ["extended by", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 228, "name": "Number of use cases inherited by this use case", "keywords": ["inherited", "use case"]},
    {"dimension": "Technical Context", "subdim_id": 229, "name": "Feature weight from use cases", "keywords": ["feature", "weight"]},
    {"dimension": "Technical Context", "subdim_id": 230, "name": "Revenue dependencies", "keywords": ["revenue", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 231, "name": "Cost dependencies", "keywords": ["cost", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 232, "name": "Inter-domain dependencies", "keywords": ["inter-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 233, "name": "Dependencies among user stories", "keywords": ["user stories", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 234, "name": "Dependencies among delivery stories", "keywords": ["delivery", "stories"]},
    {"dimension": "Technical Context", "subdim_id": 235, "name": "Intra-domain dependencies", "keywords": ["intra-domain", "dependencies"]},
    {"dimension": "Technical Context", "subdim_id": 236, "name": "Dependencies due to downstream activities", "keywords": ["downstream", "activities"]},
    {"dimension": "Technical Context", "subdim_id": 237, "name": "Team-based dependencies", "keywords": ["team", "dependencies"]}
]


def associate_with_rotation(project, dimensions,
                            min_dims=3, max_dims=7,
                            max_ess_subs=15, min_subs_per_req=1, max_subs_per_req=7):
    # --- 1) Selección de dimensiones (idéntico) ---
    guideline_words = set()
    for g in project["guidelines"]:
        guideline_words.update(clean_text(g))

    dim_score = {}
    for sub in dimensions:
        cnt = sum(1 for kw in clean_text(sub["name"]) if kw in guideline_words)
        dim_score.setdefault(sub["dimension"], 0)
        dim_score[sub["dimension"]] += cnt

    sorted_dims = [d for d,_ in sorted(dim_score.items(), key=lambda x: x[1], reverse=True)]
    sel_dims = sorted_dims[:max_dims]
    if len(sel_dims) < min_dims:
        sel_dims = sorted_dims[:min_dims]

    candidate_subs = [s for s in dimensions if s["dimension"] in sel_dims]

    # --- 2) Preparamos TF-IDF sobre nombres de subdimensiones ---
    sub_texts = [s["name"] for s in candidate_subs]
    vectorizer = TfidfVectorizer().fit(sub_texts)
    sub_vecs = vectorizer.transform(sub_texts)

    # Definimos cuáles son las "esenciales" (top N por frecuencia)
    subdim_score = {}
    for sub in candidate_subs:
        toks = clean_text(sub["name"])
        sc = 0
        for g in project["guidelines"]:
            text = g.lower()
            sc += sum(text.count(tok) for tok in toks)
        for req in project["requirements"]:
            text = req["description"].lower()
            sc += sum(text.count(tok) for tok in toks)
        subdim_score[sub["subdim_id"]] = sc
    essential_ids = sorted(subdim_score, key=lambda k: subdim_score[k], reverse=True)[:max_ess_subs]
    essential_subs = [s for s in candidate_subs if s["subdim_id"] in essential_ids]

    # --- 3) Para cada requisito: similitudes + rotación + ventana k ---
    associations = []
    for idx, req in enumerate(project["requirements"]):
        req_vec = vectorizer.transform([req["description"]])
        sims = cosine_similarity(req_vec, sub_vecs).flatten()

        # pares (subdim obj, sim_score) solo de esenciales
        pairs = [(candidate_subs[i], sims[i])
                 for i, s in enumerate(candidate_subs)
                 if candidate_subs[i]["subdim_id"] in essential_ids]
        # orden descendente
        pairs.sort(key=lambda x: x[1], reverse=True)

        # extraemos solo los objetos subdim
        ordered_subs = [sub for sub, _ in pairs]

        # rotación:
        rot = idx % len(ordered_subs)
        rotated = ordered_subs[rot:] + ordered_subs[:rot]

        # tomamos la ventana de tamaño k
        chosen = rotated[:max_subs_per_req]

        # guardamos id + name + sim score
        chosen_info = []
        for sub in chosen:
            i = candidate_subs.index(sub)
            chosen_info.append((sub["subdim_id"],
                                sub["name"],
                                sims[i]))

        associations.append({
            "RequirementID": req["id"],
            "Requirement": req["description"],
            "Assigned": chosen_info,
            "PriorityScore": sum(sc for _,_,sc in chosen_info)
        })

    # --- 4) Impresión ---
    print(f"\nProyecto: {project['name']}")
    print(f"Guidelines: {project['guidelines']}")
    print(f"Dimensiones asociadas ({len(sel_dims)}): {sel_dims}\n")

    print("Subdimensiones esenciales (ID: Nombre):")
    for sid in essential_ids:
        name = next(s["name"] for s in dimensions if s["subdim_id"]==sid)
        print(f"  {sid}: {name}")
    print()

    print("Requisitos con sus subdimensiones y PriorityScore:\n")
    for assoc in associations:
        print(f"{assoc['RequirementID']}. {assoc['Requirement']}")
        for sid, name, sc in assoc["Assigned"]:
            print(f"    - {sid}: {name} (sim={sc:.3f})")
        print(f"  → PriorityScore: {assoc['PriorityScore']:.3f}\n")

# PJT-047
project = {
    "name": "Human Resources Management Platform",
    "Description": "A platform that allows companies to post job openings and manage applications, simplifying candidate hiring.",
    "guidelines": [
        "Real-time control",
        "Low inventory alerts",
        "Supplier integration",
        "Consumption reports",
        "Information security"
    ],
    "requirements": [
        {"id": 1, "description": "As a chef, I want to see the real-time inventory of ingredients."},
        {"id": 2, "description": "As an administrator, I want to receive alerts when the inventory is low."},
        {"id": 3, "description": "As a manager, I want ingredient consumption statistics by dish."},
        {"id": 4, "description": "As a chef, I want to schedule automatic restocking orders."},
        {"id": 5, "description": "As an administrator, I want to manage access permissions to the system."},
        {"id": 6, "description": "As a manager, I want reports on ingredient costs by period."},
        {"id": 7, "description": "As a chef, I want to view a consumption history to plan purchases."},
        {"id": 8, "description": "As an administrator, I want options for integration with point of sale systems."},
        {"id": 9, "description": "As a manager, I want to compare ingredient consumption across different shifts."},
        {"id": 10, "description": "As a chef, I want to receive order recommendations based on demand."},
        {"id": 11, "description": "As an administrator, I want to generate reports on losses due to wasted ingredients."},
        {"id": 12, "description": "As a manager, I want to see statistics of the best-selling dishes and their impact on inventory."},
        {"id": 13, "description": "As a chef, I want to adjust recipes based on ingredient availability."},
        {"id": 14, "description": "As an administrator, I want to control inventory access based on user roles."},
        {"id": 15, "description": "As a manager, I want access to profitability reports for each dish."},
        {"id": 16, "description": "As a chef, I want to receive alerts about ingredient expiration dates."},
        {"id": 17, "description": "As an administrator, I want customer satisfaction reports related to quality."},
        {"id": 18, "description": "As a manager, I want options to export inventory data."},
        {"id": 19, "description": "As a chef, I want to see the inventory required for producing each weekly menu."}
    ]
}
associate_with_rotation(project, dimensions,
                        max_ess_subs=15,
                         min_subs_per_req=1,
                        max_subs_per_req=7)


Proyecto: Human Resources Management Platform
Guidelines: ['Real-time control', 'Low inventory alerts', 'Supplier integration', 'Consumption reports', 'Information security']
Dimensiones asociadas (7): ['Operational Benefits', 'Business Context', 'Costs', 'Risks', 'System-related Benefits', 'Benefits', 'Market-related Benefits']

Subdimensiones esenciales (ID: Nombre):
  184: System value of a feature
  183: Integration to external systems
  2: Linkage to overall system goals
  172: Time to market
  52: Contribution to user task
  110: Technical risk in current system
  111: Technical risk in proposed system
  105: Impact on essential non-functional requirements
  174: Impact on Maintenance
  177: Impact on long-term Evolution
  26: Feature influence on buying decision
  56: Cost reduction of IT operation
  150: Impediment of attaining the requirement
  109: Loss of confidential data
  186: IT departments technical guidelines

Requisitos con sus subdimensiones y PriorityScore:

1. As 

In [64]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [63]:
# Reemplaza <TOKEN> y TU_USUARIO
!git clone https://<TOKEN>@github.com/PFernandaP/req-prioritization.git
%cd req-prioritization


Cloning into 'req-prioritization'...


trabajamos para usted uwu
